# **CAM - Projeto 2**

# **Análise Exploratória de Dados (EDA) do conjunto de dados do grupo HSL**

## **1) Importando Bibliotecas**

In [1]:
# Universidade de Brasília - Programa de Pós-Graduação em Sistemas Mecatrônicos (PPMEC)
# Disciplina: Cibernética e Aprendizagem de Máquina
# Período: 1/2020
# Professor: Díbio Leandro Borges
# Aluno: Alceu Bernardes Castanheira de Farias 
# Matrícula : 19/0144670
#
# O projeto visa avaliar a possibilidade de diagnóstico Covid-19, não Covid-19 e pneumonias através de
# exames de sangue rotineiros. Essa classificação diagnóstico deve ser feita com o algoritmo de
# “Random Forest”. Os resultados devem estabelecer além de acurácia um ordenamento sobre quais
# variáveis de exames seriam mais relevantes.
#
# Este notebook implementa a etapa de Exploratory Data Analysis (EDA) no conjunto de dados referente ao
# grupo do Hospital Sírio Libanês (HSL), preparando o conjunto de dados e tratando os mesmos para aplicação
# dos algoritmos de Random Forest no notebook 'CAM_Projeto_2_HSL_RF.ipynb'
#
# Importando bibliotecas para realizar a Análise Exploratória dos Dados:
#
# Pandas
# Numpy
# Seaborn
# Matlplotlib
#
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## **2) Análise das informações dos pacientes**

In [2]:
# Importando dados dos pacientes do arquvio 'HSL_Pacientes_2.csv'. O caracter de separação das colunas no arquivo .csv
# é o '|'. Exibindo os 5 primeiros dados para verificar se as informações foram importadas corretamente.
#
data_pacientes_HSL = pd.read_csv('HSL_Pacientes_2.csv', sep='|')
data_pacientes_HSL.head(5)

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO,CD_PAIS,CD_UF,CD_MUNICIPIO,CD_CEPREDUZIDO
0,3487791F44C34B421C932DC8616A8437,M,1963,BR,SP,MMMM,CCCC
1,0AD1FFA4419472256666A3445414F1F9,M,1969,BR,SP,SAO PAULO,CCCC
2,BE35D08CF3EF6F114E9935F6D72C49FA,M,1964,BR,SP,SAO PAULO,CCCC
3,962F4020D456AAB602B356E57238EF42,F,1959,BR,SP,SAO PAULO,CCCC
4,F983089DAC62E42124227C856AE3444C,M,1947,BR,SP,MMMM,CCCC


In [3]:
# Vamos verificar a quantidade de pacientes no DataFrame:
print('A quantidade total de pacientes no DataFrame analisado é de: {} pacientes '.format(data_pacientes_HSL.shape[0]))
print('Cada paciente possui {} características atreladas'.format(data_pacientes_HSL.shape[1]))

A quantidade total de pacientes no DataFrame analisado é de: 4273 pacientes 
Cada paciente possui 7 características atreladas


#### **2.1) Analisando 'CD_PAIS' e 'CD_MUNICIPIO'**

In [4]:
# Vamos primeiro tratar as informações dos pacientes. É provável que a grande maioria dos pacientes sejam brasileiros, 
# mas vamos conferir se há alguma nacionalidade diferente presente nos dados
data_pacientes_HSL['CD_PAIS'].value_counts()

BR    4133
XX     140
Name: CD_PAIS, dtype: int64

In [5]:
# Temos 140 pacientes com nacionalidade não informada por algum motivo de manutenção do anonimato. Isso corresponde a
# aproximadamente 3.28% dos pacientes nos dados. A princípio, vamos desconsiderar essa coluna no projeto.
#
# Agora, analisamos a coluna 'CD_CEPREDUZIDO'. 
data_pacientes_HSL['CD_CEPREDUZIDO'].value_counts()

CCCC     4268
12460       5
Name: CD_CEPREDUZIDO, dtype: int64

In [6]:
# A maioria dos CEPs reduzidos foram omitidos. Podemos desconsiderar essa coluna, até porque contamos com informação 
# de Unidade da Federação (CD_UF) e município (CD_MUNICIPIO) que já representam o endereço dos pacientes.
#
# Com base nessas informações e nas features analisadas em (Bao, 2020), vamos considerar como features de cada 
# paciente:
#
# ID_PACIENTE = ID anonimizado de cada paciente
# IC_SEXO
# IC_IDADE = Vamos calcular a diferença entre o ano de nascimento e o ano atual para termos a idade dos pacientes
#
# Portanto, primeiramente iremos eliminar as colunas CD_PAIS e CD_CEPREDUZIDO
data_pacientes_HSL.drop(['CD_PAIS', 'CD_UF', 'CD_MUNICIPIO', 'CD_CEPREDUZIDO'], axis=1, inplace=True)
data_pacientes_HSL.head(5)

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO
0,3487791F44C34B421C932DC8616A8437,M,1963
1,0AD1FFA4419472256666A3445414F1F9,M,1969
2,BE35D08CF3EF6F114E9935F6D72C49FA,M,1964
3,962F4020D456AAB602B356E57238EF42,F,1959
4,F983089DAC62E42124227C856AE3444C,M,1947


###**2.2) Analisando a coluna 'IC_SEXO'**

In [7]:
# Com as colunas remanescente, podemos analisar se há dados faltantes para as variáveis que sobram.
data_pacientes_HSL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273 entries, 0 to 4272
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID_PACIENTE    4273 non-null   object
 1   IC_SEXO        4271 non-null   object
 2   AA_NASCIMENTO  4273 non-null   object
dtypes: object(3)
memory usage: 100.3+ KB


In [8]:
# Temos 2 dados nulos (ou que não foram disponibilziados por motivo de anonimato) nos nossos dados para a coluna
# 'IC_SEXO'; 32 dados nulos para 'CD_UF' e 1 dado nulo para 'CD_MUNICIPIO'. Podemos verificar quais são esses 
# dados nulos por meio do método isnull() para IC_SEXO_NULL, primeiramente.
IC_SEXO_NULL = (data_pacientes_HSL['IC_SEXO'].isnull() == True)
data_pacientes_HSL[IC_SEXO_NULL]

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO
1078,FD1EB8358E46C1C19F4BEE32B4203733,NaN,YYYY
1561,F72449DD367B20A2E0619201D7A14813,NaN,YYYY


In [9]:
# Como são somente dois dados faltantes, vamos preencher os mesmos com o sexo que for mais presente nos dados:
data_pacientes_HSL['IC_SEXO'].value_counts()

F    2157
M    2114
Name: IC_SEXO, dtype: int64

In [10]:
# Os dados estão bem definidos: se colocarmos o sexo como sendo 'F' a quantidade de pacientes do sexo feminino
# no DataFrame vai de aproximadamente 50.48% para aproximadamente 50.53%. Se adotarmos os dados nulos como 
# sendo 'M', a quantidade de pacientes do sexo masculino vai de aproximadamente 49.47% para aproximadamente 49.52%.
# Essa diferença parece bem pequena, então vamos continuar completar os dados faltantes com 'IC_SEXO' = 'F',
# conforme planejado inicialmente.
data_pacientes_HSL['IC_SEXO'].fillna('F', inplace=True)
data_pacientes_HSL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273 entries, 0 to 4272
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID_PACIENTE    4273 non-null   object
 1   IC_SEXO        4273 non-null   object
 2   AA_NASCIMENTO  4273 non-null   object
dtypes: object(3)
memory usage: 100.3+ KB


In [11]:
# Verificando os pacientes cujos dados de 'IC_SEXO' estavam nulos:
IC_SEXO_NULL = (data_pacientes_HSL['ID_PACIENTE'] == 'FD1EB8358E46C1C19F4BEE32B4203733') | (data_pacientes_HSL['ID_PACIENTE'] == 'F72449DD367B20A2E0619201D7A14813')
data_pacientes_HSL[IC_SEXO_NULL]

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO
1078,FD1EB8358E46C1C19F4BEE32B4203733,F,YYYY
1561,F72449DD367B20A2E0619201D7A14813,F,YYYY


In [12]:
# Vamos codificar a variável 'IC_SEXO' com uma classificação binária, para que os algoritmos de 
# floresta aleatória no próximo notebook possam tratá-los de maneira mais adequada:
#
# M = 0
# F = 1
#
IC_SEXO_map = {
    'M' : 0, 
    'F' : 1
}

# Mapeando os novos valores e verificando o dataframe resultante
data_pacientes_HSL['IC_SEXO'] = data_pacientes_HSL['IC_SEXO'].map(IC_SEXO_map)
data_pacientes_HSL.head(5)

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO
0,3487791F44C34B421C932DC8616A8437,0,1963
1,0AD1FFA4419472256666A3445414F1F9,0,1969
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,1964
3,962F4020D456AAB602B356E57238EF42,1,1959
4,F983089DAC62E42124227C856AE3444C,0,1947


###**2.3) Analisando a coluna 'AA_NASCIMENTO' e criando coluna 'IC_IDADE'**

In [13]:
# Vamos tratar agora os dados de ano de nascimento. Não temos dados nulos, mas pelos exemplos acima é possível 
# perceber que nem todos os dados informados são válidos
AA_NASCIMENTO_INVALID = data_pacientes_HSL['AA_NASCIMENTO'] == 'YYYY'
data_pacientes_HSL[AA_NASCIMENTO_INVALID]

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO
177,E16CEA2609E231B1DD6DAB162F5AF6B7,1,YYYY
183,D784FBF7EAF7E9AF8F2F4329DDCDA74F,0,YYYY
214,02F2F8FC72B4714F5C6B98C04EDB5913,1,YYYY
245,3BF2F196BC7E5CF435BFE25348AD38D6,1,YYYY
300,1805364D2C28B95984EE504720049113,1,YYYY
...,...,...,...
4021,2519B119A900443F5D5B1E858420FA96,0,YYYY
4051,28BBF1CC9F9D4E0A56E6025F706FDCA5,1,YYYY
4160,65BD29C84E392B66F53C56E0C767838A,1,YYYY
4194,115D4E83D9FEFDF16C12D9A468ECFF24,0,YYYY


In [14]:
# Temos 86 dados de idade não informados, o que corresponde a aproximadamente 2.01% dos dados. 
# Podemos completar esses dados de idade com a média das demais idades. Para tal, é necessário 
# primeiro calcular a idade dos pacientes com base no ano atual e no ano de nascimento (já que 
# não temos a data precisa, para saber se o paciente já fez aniversário esse ano, utilizamos
# somente o ano de nascimento). Vamos primeiro obter o ano atual por meio do método 
# datetime.today().year do pacote datetime. Com isso, podemos subtrair o ano atual do ano de 
# nascimento, que deve ser convertido para um valor numérico para que a operação possa ser feita.
from datetime import datetime
ano_atual = datetime.today().year

data_pacientes_HSL['IC_IDADE'] = (ano_atual - pd.to_numeric(data_pacientes_HSL['AA_NASCIMENTO'], errors='coerce'))
data_pacientes_HSL.head(5)

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO,IC_IDADE
0,3487791F44C34B421C932DC8616A8437,0,1963,57.0
1,0AD1FFA4419472256666A3445414F1F9,0,1969,51.0
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,1964,56.0
3,962F4020D456AAB602B356E57238EF42,1,1959,61.0
4,F983089DAC62E42124227C856AE3444C,0,1947,73.0


In [15]:
# O resultado da operação anterior torna os dados não numéricos em NaN:
IC_IDADE_NULL = data_pacientes_HSL['IC_IDADE'].isnull()
data_pacientes_HSL[IC_IDADE_NULL]

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO,IC_IDADE
9,87D0DF35EA2D504FA30F81E84ACB8EF4,0,AAAA,NaN
14,EF76A95879354195A081BAAE73BC980D,0,AAAA,NaN
101,2417AF2D8A05E7DF5E8D917E0D363DEC,1,AAAA,NaN
177,E16CEA2609E231B1DD6DAB162F5AF6B7,1,YYYY,NaN
183,D784FBF7EAF7E9AF8F2F4329DDCDA74F,0,YYYY,NaN
...,...,...,...,...
4215,0D9CCA6B84A642C9AAF3A6978942FC9B,1,AAAA,NaN
4246,4A605023FFBDA5EA2644B1FAAE26D23B,0,AAAA,NaN
4250,C38399695B7789855C11FD9499F3E038,1,AAAA,NaN
4266,F89078F457311BFD7FEB7EFB5420ED84,0,YYYY,NaN


In [16]:
# Vamos calcular a média de idade disponível na coluna 'IC_IDADE' para substituir os dados faltantes
data_pacientes_HSL['IC_IDADE'].mean()

43.50446751992273

In [17]:
# Vamos arredondar a média de idade para 40, já que idade como casas decimais não é comum. Substituimos
# os valores nulos da coluna 'IC_IDADE' por 44.
data_pacientes_HSL['IC_IDADE'].fillna(44, inplace=True)
data_pacientes_HSL.head(10)

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO,IC_IDADE
0,3487791F44C34B421C932DC8616A8437,0,1963,57.0
1,0AD1FFA4419472256666A3445414F1F9,0,1969,51.0
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,1964,56.0
3,962F4020D456AAB602B356E57238EF42,1,1959,61.0
4,F983089DAC62E42124227C856AE3444C,0,1947,73.0
5,FBACC8550F2B875BE5FCEBD320CF0DEF,0,1959,61.0
6,74C6B0CEF2B73BBA9AC5310A4BEC5B4D,0,1969,51.0
7,A2D9A2C3A47BEF62661458BA7B014771,0,1991,29.0
8,11CE6296257A354E5D0B342F37F82111,1,1954,66.0
9,87D0DF35EA2D504FA30F81E84ACB8EF4,0,AAAA,44.0


In [18]:
# Conferindo que não há dados faltantes em termos de idade no Dataframe
IC_IDADE_NULL = data_pacientes_HSL['IC_IDADE'].isnull()
data_pacientes_HSL[IC_IDADE_NULL]

,ID_PACIENTE,IC_SEXO,AA_NASCIMENTO,IC_IDADE


### **2.4) Excluindo variáveis e ordenando o Dataframe**

In [19]:
# Para município, temos muitos dados em anonimato. É de se supor que a maioria seja de SP, mas se colocássemos
# todos esses dados como São Paulo, teriamos mais municípios de São Paulo do que o próprio estado 'SP'. Como não 
# há uma forma clara de distribuir esses municípios, uma vez que temos uma variedade grande de estados presente
# no dataframe, optamos por excluir essa infomração, permanecendo somente com 'CD_UF' referente ao endereço
# do paciente. Vamos também eliminar a coluna 'AA_NASCIMENTO', pois já temos a idade dos pacientes na coluna 
# 'IC_IDADE'.
data_pacientes_HSL.drop(['AA_NASCIMENTO'], axis=1, inplace=True)
data_pacientes_HSL.head(5)

,ID_PACIENTE,IC_SEXO,IC_IDADE
0,3487791F44C34B421C932DC8616A8437,0,57.0
1,0AD1FFA4419472256666A3445414F1F9,0,51.0
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,56.0
3,962F4020D456AAB602B356E57238EF42,1,61.0
4,F983089DAC62E42124227C856AE3444C,0,73.0


In [20]:
# Trocando a ordem das colunas para uma estrutura mais lógica
data_pacientes_HSL = data_pacientes_HSL[['ID_PACIENTE','IC_SEXO','IC_IDADE']]
print(data_pacientes_HSL.shape[0])
data_pacientes_HSL.head(10)

4273


,ID_PACIENTE,IC_SEXO,IC_IDADE
0,3487791F44C34B421C932DC8616A8437,0,57.0
1,0AD1FFA4419472256666A3445414F1F9,0,51.0
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,56.0
3,962F4020D456AAB602B356E57238EF42,1,61.0
4,F983089DAC62E42124227C856AE3444C,0,73.0
5,FBACC8550F2B875BE5FCEBD320CF0DEF,0,61.0
6,74C6B0CEF2B73BBA9AC5310A4BEC5B4D,0,51.0
7,A2D9A2C3A47BEF62661458BA7B014771,0,29.0
8,11CE6296257A354E5D0B342F37F82111,1,66.0
9,87D0DF35EA2D504FA30F81E84ACB8EF4,0,44.0


In [21]:
# Salvando o arquivo com informações dos pacientes em formato .csv
data_pacientes_HSL.to_csv('HSL_Pacientes_2_EDA.csv')

##**3) Análise das informações de exames**

In [22]:
# Importando dados dos exames do arquvio 'HSL_Exames_2.csv'. O caracter de separação das colunas no arquivo .csv
# é o '|'. Exibindo os 5 primeiros dados para verificar se as informações foram importadas corretamente.
#
data_exames_HSL = pd.read_csv('HSL_Exames_2.csv', sep='|')
data_exames_HSL.head(5)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
0,3487791F44C34B421C932DC8616A8437,F924F6CEA1539FEFDCE7C252DFA5AF99,2020-05-14,Pronto Socorro,Fosfatase Alcalina,Fosfatase Alcalina,106,U/L,40 - 129
1,3487791F44C34B421C932DC8616A8437,F924F6CEA1539FEFDCE7C252DFA5AF99,2020-05-14,Pronto Socorro,Gama Gt,Gama-GT,33,U/L,12 a 73
2,3487791F44C34B421C932DC8616A8437,F924F6CEA1539FEFDCE7C252DFA5AF99,2020-05-14,Pronto Socorro,Tgp,ALT (TGP),51,U/L,Até 41
3,3487791F44C34B421C932DC8616A8437,F924F6CEA1539FEFDCE7C252DFA5AF99,2020-05-14,Pronto Socorro,Desidrogenase Lática (DHL/L),DHL,530,U/L,240 a 480
4,3487791F44C34B421C932DC8616A8437,F924F6CEA1539FEFDCE7C252DFA5AF99,2020-05-14,Pronto Socorro,"Proteína C Reativa, plasma",Proteína C-Reativa,"1,84",mg/dL,Ver resultado tradicional


In [23]:
#data_exames_HSL.to_csv('HSL_Exames_2_dot.csv', sep='|', decimal = '.')
#data_exames_HSL = pd.read_csv('HSL_Exames_2_dot.csv', sep='|', decimal='.')
#data_exames_HSL.head(5)

In [24]:
# Exibindo informações referentes ao tamanho dos DataFrames importados
print('O número total de exames realziados é igual a {}'.format(data_exames_HSL.shape[0]))
print('Cada exame possui {} características atreladas'.format(data_exames_HSL.shape[1]))

O número total de exames realziados é igual a 259440
Cada exame possui 9 características atreladas


### **3.1) Procurando features de interesse em 'DE_ANALITO'**

In [25]:
# Temos muito mais exames realizados do que o número total de pacientes (4273). Isso ocorre porque cada paciente
# pode realizar vários exames, que analizam diversas características, chamadas de analitos. Cada exame analisa um
# analito por vez, retornando um valor numérico ou uma string indicando o resultado do exame para o analito 
# específico. Um exame analisa um analito por vez, mas no total, vários analitos podem ser examinados com um único
# tipo de exame.
#
# De acordo com o artigo de referência 'BAO, Forrest Sheng et al. Triaging moderate COVID-19 and other viral pneumonias from routine
# blood tests', para determinação de COVID-19 ou não nos pacientes, as features mais relevantes utilizadas foram:
#
# Feature do artigo | DE_ANALITO
#
# WBC (White Blood Cells) = Leucócitos
# HGB = Hemoglobina 
# platelet = Plaquetas 
# neutrophil % | Neutrófilos (%) 
# neutrophil # | Neutrófilos 
# lymphocyte % | Linfócitos 
# lymphocyte # | Linfócitos 
# CRP (C-Reactive Protein) | Proteína C-Reativa 
# TBil | Bilirrubina Total* 
# BUN (Blood Urea Nitrogen) | Uréia 
# creatinine | Creatinina 
# LDH  (Desidrogenase Láctica) | DHL 
# D-dimer | Dimeros D, quant 
# age 
# gender 
#
# Idade e gênero dos pacientes já são informações tratados no DataFrame anterior (incluimos também UF para verificar se
# tem algum impacto nos algoritmos de aprendizagem de máquina). No conjunto de dados de exames (data_exames_HLS), 
# precisamos encontrar nos analitos ('DE_ANALITO') as características acima. Para isso, procuramos todos os diferentes
# tipos de analitos presentes no conjunto de dados de exames. Com a lista em mãos, copiamos as informações e procuramos
# os analitos especificados no artigo citado anteriormente.
data_exames_HSL['DE_ANALITO'].unique()

array(['Fosfatase Alcalina', 'Gama-GT', 'ALT (TGP)', 'DHL',
       'Proteína C-Reativa', 'AST (TGO)', 'Fibrinogenio',
       'Dimeros D, quant', 'Hemoglobina', 'VCM', 'Leucócitos',
       'Neutrófilos', 'Eosinófilos', 'Basófilos', 'Linfócitos',
       'Monócitos', 'Plaquetas', 'Eritrócitos', 'Hematócrito', 'HCM',
       'CHCM', 'RDW', 'Neutrófilos (%)', 'Eosinófilos (%)',
       'Basófilos (%)', 'Linfócitos (%)', 'Monócitos (%)',
       'Morfologia, SVE', 'Morfologia, SB', 'Troponina', 'Uréia',
       'Creatinina', 'Cálculo p/afrodescendente CKD-EPI',
       'Cálculo p/não afrodescendente CKD-EPI',
       'Cálculo p/afrodescendente MDRD',
       'Cálculo p/não afrodescendente MDRD', 'Potássio', 'Sódio', 'CK',
       'Volume plaquetário médio', 'Fração Imatura de Plaquetas',
       'Covid 19, Anticorpos IgG, Quimiolumin.-Índice',
       'Covid 19, Anticorpos IgG, Quimioluminescência',
       'Covid 19, Anticorpos IgM, Quimiolumin.-Índice',
       'Covid 19, Anticorpos IgM, Quimiolumines

### **3.2) Procurando features de interesse em 'DE_EXAME'**

In [26]:
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Como existe uma gama grande de tipos de exame realizados ('DE_EXAME'), é possível que um tipo de analito seja medido
# em exames diferentes. Por isso, precisamos filtrar os tipos de exames disponíveis para encontrar qual o exame que 
# será avaliado.
data_exames_HSL['DE_EXAME'].unique()

array(['Fosfatase Alcalina', 'Gama Gt', 'Tgp',
       'Desidrogenase Lática (DHL/L)', 'Proteína C Reativa, plasma',
       'Tgo', 'Fibrinogênio, Dosagem Do',
       'D. Dimeros(Dimeros De Fibrina)', 'Hemograma', 'Troponina',
       'Uréia',
       'Calculo da Estimativa da Taxa, de Filtracao Glomular, soro',
       'Creatinina', 'Potássio', 'Sódio', 'Creatinofosfoquinase Cpk-Ck',
       'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro',
       'Hemograma, sangue total',
       'Proteína C Reativa, por imunoturbidimetria, soro',
       'Desidrogenase Lática  ( LDH )', 'Hemoglobina Glicada',
       'COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)',
       'Detecção de Coronavírus (NCoV-2019) POR PCR (Anatomia Patológica)',
       'Interleucina 6, soro', 'T4L, livre, soro',
       'Bacilos Gram-Negativos Resistentes a Carbapenêmicos, Cultura, Vários Materiais',
       'Urina, Cultura - Jato Médio', 'Urina Tipo I - Jato Medio',
       'Tempo E Ativid. Protrombina (Tp)', '

### **3.3) Analisando Leucócitos**

In [27]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Leucócitos') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5119


In [28]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
leucocitos = data_exames_HSL[sel]
leucocitos.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
leucocitos.rename(columns={'DE_RESULTADO':'DE_LEUCOCITOS'}, inplace=True)
leucocitos.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_LEUCOCITOS
10,3487791F44C34B421C932DC8616A8437,2020-05-14,3210
47,3487791F44C34B421C932DC8616A8437,2020-05-04,3390
71,3487791F44C34B421C932DC8616A8437,2020-05-11,2850
128,3487791F44C34B421C932DC8616A8437,2020-05-08,2860
207,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,4370


In [29]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
leucocitos['DE_LEUCOCITOS'] = pd.to_numeric(leucocitos['DE_LEUCOCITOS'].str.replace(',','.'), errors='coerce')
leucocitos.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_LEUCOCITOS
10,3487791F44C34B421C932DC8616A8437,2020-05-14,3210
47,3487791F44C34B421C932DC8616A8437,2020-05-04,3390
71,3487791F44C34B421C932DC8616A8437,2020-05-11,2850
128,3487791F44C34B421C932DC8616A8437,2020-05-08,2860
207,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,4370
248,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,11360
279,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,7670
326,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,7380
376,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,5990
400,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,14670


In [30]:
# Verificando dados faltantes
leucocitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5119 entries, 10 to 259411
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID_PACIENTE    5119 non-null   object
 1   DT_COLETA      5119 non-null   object
 2   DE_LEUCOCITOS  5119 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 160.0+ KB


### **3.4) Analisando Hemoglobina**

In [31]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Hemoglobina') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5119


In [32]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
hemoglobina = data_exames_HSL[sel]
hemoglobina.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
hemoglobina.rename(columns={'DE_RESULTADO':'DE_HEMOGLOBINA'}, inplace=True)
hemoglobina.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_HEMOGLOBINA
8,3487791F44C34B421C932DC8616A8437,2020-05-14,"13,3"
45,3487791F44C34B421C932DC8616A8437,2020-05-04,"14,8"
69,3487791F44C34B421C932DC8616A8437,2020-05-11,"15,1"
126,3487791F44C34B421C932DC8616A8437,2020-05-08,"14,8"
205,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,"14,1"


In [33]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
hemoglobina['DE_HEMOGLOBINA'] = pd.to_numeric(hemoglobina['DE_HEMOGLOBINA'].str.replace(',','.'), errors='coerce')
hemoglobina.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_HEMOGLOBINA
8,3487791F44C34B421C932DC8616A8437,2020-05-14,13.3
45,3487791F44C34B421C932DC8616A8437,2020-05-04,14.8
69,3487791F44C34B421C932DC8616A8437,2020-05-11,15.1
126,3487791F44C34B421C932DC8616A8437,2020-05-08,14.8
205,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,14.1
246,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,15.0
277,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,13.8
324,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,13.4
374,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,16.1
398,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,15.6


In [34]:
# Verificando dados faltantes
hemoglobina.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5119 entries, 8 to 259409
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID_PACIENTE     5119 non-null   object 
 1   DT_COLETA       5119 non-null   object 
 2   DE_HEMOGLOBINA  5119 non-null   float64
dtypes: float64(1), object(2)
memory usage: 160.0+ KB


### **3.5) Analisando Plaquetas**

In [35]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Plaquetas') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5118


In [36]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
plaquetas = data_exames_HSL[sel]
plaquetas.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
plaquetas.rename(columns={'DE_RESULTADO':'DE_PLAQUETAS'}, inplace=True)
plaquetas.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_PLAQUETAS
16,3487791F44C34B421C932DC8616A8437,2020-05-14,97000
53,3487791F44C34B421C932DC8616A8437,2020-05-04,86000
77,3487791F44C34B421C932DC8616A8437,2020-05-11,79000
134,3487791F44C34B421C932DC8616A8437,2020-05-08,84000
213,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,201000


In [37]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
plaquetas['DE_PLAQUETAS'] = pd.to_numeric(plaquetas['DE_PLAQUETAS'].str.replace(',','.'), errors='coerce')
plaquetas.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_PLAQUETAS
16,3487791F44C34B421C932DC8616A8437,2020-05-14,97000.0
53,3487791F44C34B421C932DC8616A8437,2020-05-04,86000.0
77,3487791F44C34B421C932DC8616A8437,2020-05-11,79000.0
134,3487791F44C34B421C932DC8616A8437,2020-05-08,84000.0
213,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,201000.0
254,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,220000.0
285,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,218000.0
332,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,215000.0
382,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,323000.0
406,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,414000.0


In [38]:
# Verificando dados faltantes
plaquetas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5118 entries, 16 to 259417
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID_PACIENTE   5118 non-null   object 
 1   DT_COLETA     5118 non-null   object 
 2   DE_PLAQUETAS  5111 non-null   float64
dtypes: float64(1), object(2)
memory usage: 159.9+ KB


In [39]:
# Temos dados que não representam valores numéricos (NaN). Vamos substituir esses dados pela 
# média dos demais dados da coluna, mesma estratégia utilizada em (Bao, 2020):
plaquetas.fillna(plaquetas.mean(), inplace=True)
plaquetas.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,ID_PACIENTE,DT_COLETA,DE_PLAQUETAS
16,3487791F44C34B421C932DC8616A8437,2020-05-14,97000.0
53,3487791F44C34B421C932DC8616A8437,2020-05-04,86000.0
77,3487791F44C34B421C932DC8616A8437,2020-05-11,79000.0
134,3487791F44C34B421C932DC8616A8437,2020-05-08,84000.0
213,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,201000.0
254,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,220000.0
285,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,218000.0
332,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,215000.0
382,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,323000.0
406,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,414000.0


In [40]:
# Verificando dados faltantes
plaquetas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5118 entries, 16 to 259417
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID_PACIENTE   5118 non-null   object 
 1   DT_COLETA     5118 non-null   object 
 2   DE_PLAQUETAS  5118 non-null   float64
dtypes: float64(1), object(2)
memory usage: 159.9+ KB


### **3.6) Analisando Neutrófilos (%)**

In [41]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Neutrófilos (%)') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5115


In [42]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
neutrofilos_percent = data_exames_HSL[sel]
neutrofilos_percent.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
neutrofilos_percent.rename(columns={'DE_RESULTADO':'DE_NEUTROFILOS (%)'}, inplace=True)
neutrofilos_percent.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_NEUTROFILOS (%)
22,3487791F44C34B421C932DC8616A8437,2020-05-14,"58,6"
59,3487791F44C34B421C932DC8616A8437,2020-05-04,"62,8"
83,3487791F44C34B421C932DC8616A8437,2020-05-11,"55,4"
140,3487791F44C34B421C932DC8616A8437,2020-05-08,"59,5"
219,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,"83,6"


In [43]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
neutrofilos_percent['DE_NEUTROFILOS (%)'] = pd.to_numeric(neutrofilos_percent['DE_NEUTROFILOS (%)'].str.replace(',','.'), errors='coerce')
neutrofilos_percent.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_NEUTROFILOS (%)
22,3487791F44C34B421C932DC8616A8437,2020-05-14,58.6
59,3487791F44C34B421C932DC8616A8437,2020-05-04,62.8
83,3487791F44C34B421C932DC8616A8437,2020-05-11,55.4
140,3487791F44C34B421C932DC8616A8437,2020-05-08,59.5
219,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,83.6
260,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,91.3
291,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,85.2
338,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,86.4
388,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,59.7
412,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,56.2


In [44]:
# Verificando dados faltantes
neutrofilos_percent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5115 entries, 22 to 259423
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PACIENTE         5115 non-null   object 
 1   DT_COLETA           5115 non-null   object 
 2   DE_NEUTROFILOS (%)  5115 non-null   float64
dtypes: float64(1), object(2)
memory usage: 159.8+ KB


### **3.7) Analisando Neutrófilos**

In [45]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Neutrófilos') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5116


In [46]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
neutrofilos = data_exames_HSL[sel]
neutrofilos.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
neutrofilos.rename(columns={'DE_RESULTADO':'DE_NEUTROFILOS'}, inplace=True)
neutrofilos.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_NEUTROFILOS
11,3487791F44C34B421C932DC8616A8437,2020-05-14,1880
48,3487791F44C34B421C932DC8616A8437,2020-05-04,2130
72,3487791F44C34B421C932DC8616A8437,2020-05-11,1580
129,3487791F44C34B421C932DC8616A8437,2020-05-08,1700
208,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,3650


In [47]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
neutrofilos['DE_NEUTROFILOS'] = pd.to_numeric(neutrofilos['DE_NEUTROFILOS'].str.replace(',','.'), errors='coerce')
neutrofilos.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_NEUTROFILOS
11,3487791F44C34B421C932DC8616A8437,2020-05-14,1880
48,3487791F44C34B421C932DC8616A8437,2020-05-04,2130
72,3487791F44C34B421C932DC8616A8437,2020-05-11,1580
129,3487791F44C34B421C932DC8616A8437,2020-05-08,1700
208,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,3650
249,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,10370
280,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,6540
327,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,6370
377,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,3570
401,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,8250


In [48]:
# Verificando dados faltantes
neutrofilos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5116 entries, 11 to 259412
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID_PACIENTE     5116 non-null   object
 1   DT_COLETA       5116 non-null   object
 2   DE_NEUTROFILOS  5116 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 159.9+ KB


### **3.8) Analisando Linfócitos (%)**

In [49]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Linfócitos (%)') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5114


In [50]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
linfocitos_percent = data_exames_HSL[sel]
linfocitos_percent.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
linfocitos_percent.rename(columns={'DE_RESULTADO':'DE_LINFOCITOS (%)'}, inplace=True)
linfocitos_percent.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_LINFOCITOS (%)
25,3487791F44C34B421C932DC8616A8437,2020-05-14,"24,6"
62,3487791F44C34B421C932DC8616A8437,2020-05-04,"14,5"
86,3487791F44C34B421C932DC8616A8437,2020-05-11,"29,5"
143,3487791F44C34B421C932DC8616A8437,2020-05-08,"26,9"
222,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,"12,1"


In [51]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
linfocitos_percent['DE_LINFOCITOS (%)'] = pd.to_numeric(linfocitos_percent['DE_LINFOCITOS (%)'].str.replace(',','.'), errors='coerce')
linfocitos_percent.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_LINFOCITOS (%)
25,3487791F44C34B421C932DC8616A8437,2020-05-14,24.6
62,3487791F44C34B421C932DC8616A8437,2020-05-04,14.5
86,3487791F44C34B421C932DC8616A8437,2020-05-11,29.5
143,3487791F44C34B421C932DC8616A8437,2020-05-08,26.9
222,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,12.1
263,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,4.8
294,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,8.5
341,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,7.3
391,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,24.0
415,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,32.6


In [52]:
# Verificando dados faltantes
linfocitos_percent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5114 entries, 25 to 259426
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID_PACIENTE        5114 non-null   object 
 1   DT_COLETA          5114 non-null   object 
 2   DE_LINFOCITOS (%)  5114 non-null   float64
dtypes: float64(1), object(2)
memory usage: 159.8+ KB


### **3.9) Analisando Linfócitos**

In [53]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Linfócitos') & (data_exames_HSL['DE_EXAME'] == 'Hemograma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 5114


In [54]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
linfocitos = data_exames_HSL[sel]
linfocitos.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
linfocitos.rename(columns={'DE_RESULTADO':'DE_LINFOCITOS'}, inplace=True)
linfocitos.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_LINFOCITOS
14,3487791F44C34B421C932DC8616A8437,2020-05-14,790
51,3487791F44C34B421C932DC8616A8437,2020-05-04,490
75,3487791F44C34B421C932DC8616A8437,2020-05-11,840
132,3487791F44C34B421C932DC8616A8437,2020-05-08,770
211,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,530


In [55]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
linfocitos['DE_LINFOCITOS'] = pd.to_numeric(linfocitos['DE_LINFOCITOS'].str.replace(',','.'), errors='coerce')
linfocitos.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_LINFOCITOS
14,3487791F44C34B421C932DC8616A8437,2020-05-14,790
51,3487791F44C34B421C932DC8616A8437,2020-05-04,490
75,3487791F44C34B421C932DC8616A8437,2020-05-11,840
132,3487791F44C34B421C932DC8616A8437,2020-05-08,770
211,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,530
252,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,550
283,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,650
330,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,540
380,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,1440
404,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,4780


In [56]:
# Verificando dados faltantes
linfocitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5114 entries, 14 to 259415
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID_PACIENTE    5114 non-null   object
 1   DT_COLETA      5114 non-null   object
 2   DE_LINFOCITOS  5114 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 159.8+ KB


### **3.10) Analisando Proteína C-Reativa**

In [57]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Proteína C-Reativa') & (data_exames_HSL['DE_EXAME'] == 'Proteína C Reativa, plasma')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 4857


In [58]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
proteina_c = data_exames_HSL[sel]
proteina_c.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
proteina_c.rename(columns={'DE_RESULTADO':'DE_PROTEINA_C_REATIVA'}, inplace=True)
proteina_c.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_PROTEINA_C_REATIVA
4,3487791F44C34B421C932DC8616A8437,2020-05-14,"1,84"
34,3487791F44C34B421C932DC8616A8437,2020-05-04,"0,19"
68,3487791F44C34B421C932DC8616A8437,2020-05-11,"0,22"
120,3487791F44C34B421C932DC8616A8437,2020-05-08,"0,30"
268,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,"15,58"


In [59]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
proteina_c['DE_PROTEINA_C_REATIVA'] = pd.to_numeric(proteina_c['DE_PROTEINA_C_REATIVA'].str.replace(',','.'), errors='coerce')
proteina_c.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_PROTEINA_C_REATIVA
4,3487791F44C34B421C932DC8616A8437,2020-05-14,1.84
34,3487791F44C34B421C932DC8616A8437,2020-05-04,0.19
68,3487791F44C34B421C932DC8616A8437,2020-05-11,0.22
120,3487791F44C34B421C932DC8616A8437,2020-05-08,0.30
268,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,15.58
304,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,9.09
319,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,6.35
368,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,0.36
423,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,0.85
438,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,0.14


In [60]:
# Verificando dados faltantes
proteina_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4857 entries, 4 to 259404
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID_PACIENTE            4857 non-null   object 
 1   DT_COLETA              4857 non-null   object 
 2   DE_PROTEINA_C_REATIVA  4823 non-null   float64
dtypes: float64(1), object(2)
memory usage: 151.8+ KB


In [61]:
# Temos dados que não representam valores numéricos (NaN). Vamos substituir esses dados pela 
# média dos demais dados da coluna, mesma estratégia utilizada em (Bao, 2020):
proteina_c.fillna(proteina_c.mean(), inplace=True)
proteina_c.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,ID_PACIENTE,DT_COLETA,DE_PROTEINA_C_REATIVA
4,3487791F44C34B421C932DC8616A8437,2020-05-14,1.84
34,3487791F44C34B421C932DC8616A8437,2020-05-04,0.19
68,3487791F44C34B421C932DC8616A8437,2020-05-11,0.22
120,3487791F44C34B421C932DC8616A8437,2020-05-08,0.30
268,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,15.58
304,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,9.09
319,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,6.35
368,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,0.36
423,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,0.85
438,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,0.14


In [62]:
# Verificando dados faltantes
proteina_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4857 entries, 4 to 259404
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID_PACIENTE            4857 non-null   object 
 1   DT_COLETA              4857 non-null   object 
 2   DE_PROTEINA_C_REATIVA  4857 non-null   float64
dtypes: float64(1), object(2)
memory usage: 151.8+ KB


### **3.11) Analisando Bilirrubina Total**

In [63]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Bilirrubina Total') & (data_exames_HSL['DE_EXAME'] == 'Bilirrubinas Total E Fracoes')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 1231


In [64]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
bilirrubina = data_exames_HSL[sel]
bilirrubina.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
bilirrubina.rename(columns={'DE_RESULTADO':'DE_BILIRRUBINA'}, inplace=True)
bilirrubina.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_BILIRRUBINA
181,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,"0,43"
431,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,"0,71"
563,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-27,"1,04"
708,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-05,"0,57"
968,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-08,"0,61"


In [65]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
bilirrubina['DE_BILIRRUBINA'] = pd.to_numeric(bilirrubina['DE_BILIRRUBINA'].str.replace(',','.'), errors='coerce')
bilirrubina.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_BILIRRUBINA
181,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,0.43
431,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,0.71
563,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-27,1.04
708,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-05,0.57
968,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-08,0.61
1061,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-07,0.62
1213,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-06,0.65
1246,F5B5BD59D98836D8AA2D8C27B01225AB,2020-05-29,0.46
1326,F5B5BD59D98836D8AA2D8C27B01225AB,2020-05-27,0.45
1534,F5B5BD59D98836D8AA2D8C27B01225AB,2020-05-22,0.56


In [66]:
# Verificando dados faltantes
bilirrubina.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1231 entries, 181 to 259402
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID_PACIENTE     1231 non-null   object 
 1   DT_COLETA       1231 non-null   object 
 2   DE_BILIRRUBINA  1231 non-null   float64
dtypes: float64(1), object(2)
memory usage: 38.5+ KB


### **3.12) Analisando Ureía**

In [67]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Uréia') & (data_exames_HSL['DE_EXAME'] == 'Uréia')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 4604


In [68]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
ureia = data_exames_HSL[sel]
ureia.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
ureia.rename(columns={'DE_RESULTADO':'DE_UREIA'}, inplace=True)
ureia.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_UREIA
35,3487791F44C34B421C932DC8616A8437,2020-05-04,28
197,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,21
245,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,25
303,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,27
322,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,26


In [69]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
ureia['DE_UREIA'] = pd.to_numeric(ureia['DE_UREIA'].str.replace(',','.'), errors='coerce')
ureia.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_UREIA
35,3487791F44C34B421C932DC8616A8437,2020-05-04,28
197,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,21
245,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,25
303,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,27
322,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,26
365,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,18
426,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,32
437,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,28
500,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-29,73
572,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-27,59


In [70]:
# Verificando dados faltantes
ureia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4604 entries, 35 to 259405
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_PACIENTE  4604 non-null   object
 1   DT_COLETA    4604 non-null   object
 2   DE_UREIA     4604 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 143.9+ KB


### **3.13) Analisando Creatinina**

In [71]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Creatinina') & (data_exames_HSL['DE_EXAME'] == 'Creatinina')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 4707


In [72]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
creatinina = data_exames_HSL[sel]
creatinina.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
creatinina.rename(columns={'DE_RESULTADO':'DE_CREATININA'}, inplace=True)
creatinina.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_CREATININA
41,3487791F44C34B421C932DC8616A8437,2020-05-04,"0,91"
196,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,"0,68"
244,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,"0,80"
302,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,"0,82"
321,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,"0,62"


In [73]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
creatinina['DE_CREATININA'] = pd.to_numeric(creatinina['DE_CREATININA'].str.replace(',','.'), errors='coerce')
creatinina.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_CREATININA
41,3487791F44C34B421C932DC8616A8437,2020-05-04,0.91
196,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,0.68
244,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,0.80
302,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,0.82
321,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,0.62
364,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,0.81
425,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,1.33
439,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,1.24
499,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-29,1.22
573,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-27,1.08


In [74]:
# Verificando dados faltantes
creatinina.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4707 entries, 41 to 259406
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_PACIENTE    4707 non-null   object 
 1   DT_COLETA      4707 non-null   object 
 2   DE_CREATININA  4707 non-null   float64
dtypes: float64(1), object(2)
memory usage: 147.1+ KB


### **3.14) Analisando DHL**

In [75]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'DHL') & (data_exames_HSL['DE_EXAME'] == 'Desidrogenase Lática (DHL/L)')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 1663


In [76]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
dhl = data_exames_HSL[sel]
dhl.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
dhl.rename(columns={'DE_RESULTADO':'DE_DHL'}, inplace=True)
dhl.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_DHL
3,3487791F44C34B421C932DC8616A8437,2020-05-14,530
33,3487791F44C34B421C932DC8616A8437,2020-05-04,364
123,3487791F44C34B421C932DC8616A8437,2020-05-08,347
187,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,651
300,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,645


In [77]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
dhl['DE_DHL'] = pd.to_numeric(dhl['DE_DHL'].str.replace(',','.'), errors='coerce')
dhl.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_DHL
3,3487791F44C34B421C932DC8616A8437,2020-05-14,530
33,3487791F44C34B421C932DC8616A8437,2020-05-04,364
123,3487791F44C34B421C932DC8616A8437,2020-05-08,347
187,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,651
300,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,645
318,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,540
362,B9697B4994DF6367EDB722F1EC4AB423,2020-05-18,387
420,D1C4E29D126CA8B4359827903CB222AC,2020-03-30,438
466,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-28,566
570,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-27,664


In [78]:
# Verificando dados faltantes
dhl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1663 entries, 3 to 259396
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_PACIENTE  1663 non-null   object
 1   DT_COLETA    1663 non-null   object
 2   DE_DHL       1663 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 52.0+ KB


### **3.15) Analisando Dimeros D**

In [79]:
# Agora que sabemos a combinação de analitos e exames que procuramos, precisamos entender os tipos de resultado
# obtidos para cada uma das features analisadas nos exames:
#
# DE_ANALITO | DE_EXAME
#
# Leucócitos | Hemograma
# Hemoglobina | Hemograma
# Plaquetas | Hemograma
# Neutrófilos (%) | Hemograma
# Neutrófilos | Hemograma
# Linfócitos (%) | Hemograma 
# Linfócitos | Hemograma 
# Proteína C-Reativa | Proteína C Reativa, plasma
# Bilirrubina Total | Bilirrubinas Total E Fracoes
# Uréia | Uréia 
# Creatinina | Creatinina 
# DHL | Desidrogenase Lática (DHL/L)
# Dimeros D, quant | D. Dimeros(Dimeros De Fibrina)
#
# Vamos analisar os resultados para cada conjunto de analitos e exames individualmente
sel = (data_exames_HSL['DE_ANALITO'] == 'Dimeros D, quant') & (data_exames_HSL['DE_EXAME'] == 'D. Dimeros(Dimeros De Fibrina)')
print('A quantidade de exames deste tipo realizados é de: {}'.format(data_exames_HSL[sel].shape[0]))

A quantidade de exames deste tipo realizados é de: 2475


In [80]:
# Salvando os resultados deste tipo de exame e analito em um dataframe. 
# Excluímos as colunas que não serão necessárias para o projeto e renomeamos a
# coluna 'DE_RESULTADO' para o nome do 'ANALITO' analisado
dimeros_d = data_exames_HSL[sel]
dimeros_d.drop(['ID_ATENDIMENTO', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
dimeros_d.rename(columns={'DE_RESULTADO':'DE_DIMEROS_D'}, inplace=True)
dimeros_d.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,DT_COLETA,DE_DIMEROS_D
7,3487791F44C34B421C932DC8616A8437,2020-05-14,256
30,3487791F44C34B421C932DC8616A8437,2020-05-04,248
118,3487791F44C34B421C932DC8616A8437,2020-05-25,inferior a 215
125,3487791F44C34B421C932DC8616A8437,2020-05-08,296
174,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,1406


In [81]:
# Vamos converter os valores da coluna para valores numéricos, já que os resultados dos exames estão sendo
# dados por strings. Precisamos trocar o caracter que divide a parte decimal de ',' para '.' para efetuar a
# operação. Valores não numéricos são substituidos por NaN (errors='coerce') e serão tratados mais adiante.
dimeros_d['DE_DIMEROS_D'] = pd.to_numeric(dimeros_d['DE_DIMEROS_D'].str.replace(',','.'), errors='coerce')
dimeros_d.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ID_PACIENTE,DT_COLETA,DE_DIMEROS_D
7,3487791F44C34B421C932DC8616A8437,2020-05-14,256.0
30,3487791F44C34B421C932DC8616A8437,2020-05-04,248.0
118,3487791F44C34B421C932DC8616A8437,2020-05-25,NaN
125,3487791F44C34B421C932DC8616A8437,2020-05-08,296.0
174,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,1406.0
306,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,1363.0
346,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,1003.0
463,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,256.0
468,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-28,331.0
503,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-29,343.0


In [82]:
# Verificando dados faltantes
dimeros_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2475 entries, 7 to 259434
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID_PACIENTE   2475 non-null   object 
 1   DT_COLETA     2475 non-null   object 
 2   DE_DIMEROS_D  2325 non-null   float64
dtypes: float64(1), object(2)
memory usage: 77.3+ KB


In [83]:
# Temos dados que não representam valores numéricos (NaN). Vamos substituir esses dados pela 
# média dos demais dados da coluna, mesma estratégia utilizada em (Bao, 2020):
dimeros_d.fillna(dimeros_d.mean(), inplace=True)
dimeros_d.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,ID_PACIENTE,DT_COLETA,DE_DIMEROS_D
7,3487791F44C34B421C932DC8616A8437,2020-05-14,256.000000
30,3487791F44C34B421C932DC8616A8437,2020-05-04,248.000000
118,3487791F44C34B421C932DC8616A8437,2020-05-25,1437.374194
125,3487791F44C34B421C932DC8616A8437,2020-05-08,296.000000
174,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-08,1406.000000
306,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-09,1363.000000
346,B8A474D172DDB5BAD11F088BDA20BE96,2020-08-10,1003.000000
463,D1C4E29D126CA8B4359827903CB222AC,2020-07-14,256.000000
468,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-28,331.000000
503,EA3D530A51086CEFC8C6DFAEC53D00F2,2020-03-29,343.000000


In [84]:
# Verificando dados faltantes
dimeros_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2475 entries, 7 to 259434
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID_PACIENTE   2475 non-null   object 
 1   DT_COLETA     2475 non-null   object 
 2   DE_DIMEROS_D  2475 non-null   float64
dtypes: float64(1), object(2)
memory usage: 77.3+ KB


### **3.16) Analisando os exames para COVID-19**

In [85]:
# Além dos dados específicos dos exames, queremos saber os pacientes que fizeram o exame para COVID. 
# Os exames disponíveis para detecção de COVID neste conjunto de dados são:
#
# 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro'
# 'COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)'
# 'COVID-19-Teste Rápido (IgM e IgG), soro'
# 'Detecção de Coronavírus (NCoV-2019) POR PCR (Anatomia Patológica)'
# 'Sorologia - Coronavírus, IgG'
# 'Sorologia - Coronavírus, IgA'
#
# Vamos primeiro analisar individualmente quantos exames de COVID-19 de cada tipo foram realizados
exames_covid_soro = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro')
exames_covid_pcr = (data_exames_HSL['DE_EXAME'] == 'COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)')
exames_covid_rapido = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Teste Rápido (IgM e IgG), soro')
exames_corona_pcr = (data_exames_HSL['DE_EXAME'] == 'Detecção de Coronavírus (NCoV-2019) POR PCR (Anatomia Patológica)')
exames_corona_igg = (data_exames_HSL['DE_EXAME'] == 'Sorologia - Coronavírus, IgG')
exames_corona_iga = (data_exames_HSL['DE_EXAME'] == 'Sorologia - Coronavírus, IgA')

# Exibindo o número de exames de cada tipo realizados
print('No total foram realizados {} testes do tipo COVID-19-Sorologia IgM e IgG por quimiluminescência, soro'.format(data_exames_HSL[exames_covid_soro].shape[0]))
print('No total foram realizados {} testes do tipo COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)'.format(data_exames_HSL[exames_covid_pcr].shape[0]))
print('No total foram realizados {} testes do tipo COVID-19-Teste Rápido (IgM e IgG), soro'.format(data_exames_HSL[exames_covid_rapido].shape[0]))
print('No total foram realizados {} testes do tipo COVID-19-PCR (Anatomia Patológica)'.format(data_exames_HSL[exames_corona_pcr].shape[0]))
print('No total foram realizados {} testes do tipo COVID-19-Sorologia IgG'.format(data_exames_HSL[exames_corona_igg].shape[0]))
print('No total foram realizados {} testes do tipo COVID-19-Sorologia IgA'.format(data_exames_HSL[exames_corona_iga].shape[0]))

No total foram realizados 402 testes do tipo COVID-19-Sorologia IgM e IgG por quimiluminescência, soro
No total foram realizados 3070 testes do tipo COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)
No total foram realizados 95 testes do tipo COVID-19-Teste Rápido (IgM e IgG), soro
No total foram realizados 492 testes do tipo COVID-19-PCR (Anatomia Patológica)
No total foram realizados 86 testes do tipo COVID-19-Sorologia IgG
No total foram realizados 66 testes do tipo COVID-19-Sorologia IgA


### **3.17) Análise dos testes sorológicos**

####**3.17.1) Sorologia IgM e IgG**

#####**Anticorpos IgG, Quimiolumin.-Índice**

In [86]:
# Vamos analisar quais os resultados de testes que temos no conjunto de dados, 
# quando analisamos os testes sorológicos para COVID-19.
# Primeiro, vamos analisar os resultados do exame 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro'
data_exames_HSL[exames_covid_soro]['DE_RESULTADO'].unique()

array(['50,0', 'REAGENTE', '5,6', '42,6', '1,8', '0,1', 'NÃO REAGENTE',
       '0,8', 'Indeterminado',
       'Ausência de anticorpos contra SARS-CoV-2 detectáveis nesta amostra, após análise em duas metodologias, Este perfil é compatível com ausência de infecção por este vírus, ou ausência de soroconversão no período decorrido entre a infecção e a coleta, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Desse modo, podem haver resultados negativos de sorologia mesmo em casos de doença anteriormente confirmada por PCR, Se persitir a hipótese diagnóstica, sugere-se seguimento sorológico em, no mínimo, 14 dias,',
       '0,6', '0,5', '2,2', '1,4', '0,3', '144,0', '2,0',
       'Detectados anticorpos da classe IgG contra SARS-CoV-2, Este perfil é compatível com infecção pregressa, Estudos demonstram que, sobretudo em pessoas qu

In [87]:
# Temos resultados tanto numéricos quanto categóricos, que descrevem se foi detectado COVID ou não no paciente.
# Nesse caso, precisamos analisar os analitos separadamente, caracterizando para cada um se o paciente teve ou
# não COVID. Essa análise mais detalhada é importante, pois a princípio, os testes sorológicos são mais confiáveis
# do que os testes PCR e os testes rápidos.
data_exames_HSL[exames_covid_soro]['DE_ANALITO'].unique()

array(['Covid 19, Anticorpos IgG, Quimiolumin.-Índice',
       'Covid 19, Anticorpos IgG, Quimioluminescência',
       'Covid 19, Anticorpos IgM, Quimiolumin.-Índice',
       'Covid 19, Anticorpos IgM, Quimioluminescência',
       'Covid 19, Anticorpos Totais, Eletroquim-Índic',
       'Covid 19, Anticorpos Totais, Eletroquimiolumi',
       'Covid 19, Sorologia - Interpretação',
       'Covid 19, Sorologia - Conclusão'], dtype=object)

In [88]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgG, Quimiolumin.-Índice')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['inferior a 0,8', 'inferior a 0,9'], dtype=object)

In [89]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].unique()

array(['50,0', '42,6', '0,1', '2,2', '2,0', '5,1', '0,3', '2,9', '37,3',
       '51,7', '0,5', 'superior a 100,0', '1,8', '15,6', '25,5', '8,3',
       '18,1', '11,8', '1,4', '15,1', '0,7', '50,1', '24,0', '28,4',
       '19,3', '0,2', '2,3', '27,9', '0,4', '76,4', '11,2', '6,6', '0,6',
       '21,1', '8,1', '1,0', '10,7', 'superior a 30,0', '79,6', '1,5'],
      dtype=object)

In [90]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
# Para o caso do analito em questão, vamos considerar todos os valores inferiores a 0.9 como 0
# e valores iguais ou superiores a 0.9 como 1 (com base nos valores de referência para o analito)
resultados_map = {
   '50,0' : 1, 
   '42,6' : 1, 
   '0,1' : 0, 
   '2,2' : 1, 
   '2,0' : 1, 
   '5,1' : 1, 
   '0,3' : 0, 
   '2,9' : 1, 
   '37,3' : 1,
   '51,7' : 1, 
   '0,5' : 0, 
   'superior a 100,0' : 1, 
   '1,8' : 1, 
   '15,6' : 1, 
   '25,5' : 1, 
   '8,3' : 1,
   '18,1' : 1, 
   '11,8' : 1, 
   '1,4' : 1, 
   '15,1' : 1, 
   '0,7' : 0, 
   '50,1' : 1, 
   '24,0' : 1,
   '28,4' : 1,
   '19,3' : 1, 
   '0,2' : 0, 
   '2,3' : 1, 
   '27,9' : 1, 
   '0,4' : 0, 
   '76,4' : 1, 
   '11,2' : 1, 
   '6,6' : 1, 
   '0,6' : 0,
   '21,1' : 1, 
   '8,1' : 1, 
   '1,0' : 1, 
   '10,7' : 1, 
   'superior a 30,0' : 1, 
   '79,6' : 1, 
   '1,5' : 1 
}

exames_soro_1 = data_exames_HSL[analito]
exames_soro_1['DE_RESULTADO'] = exames_soro_1['DE_RESULTADO'].map(resultados_map)
exames_soro_1.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_1.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
90,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,1
1217,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,1
2855,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,0
2864,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,0
6020,F99249272E611207DBEC7EE3CF81F0E5,69ED1FBA20A1B0755943B89F82104ED8,2020-06-04,0
8959,74D237DB6D34A5BB2A95A26CD9FD5CF9,4FCD5F94A21B633D76E728EE17C4E46C,2020-06-27,0
18267,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,626B753CB67B893001127A521FED3960,2020-06-26,0
18275,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,6D5114B9017440527D233D0DAE085F1F,2020-07-09,1
19139,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,0
24203,C16902E84856371CFB33837851722042,66AF0EFC46FC52B87617914CE4780338,2020-06-03,0


##### **Anticorpos IgG, Quimioluminescência**


In [91]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgG, Quimioluminescência')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['Não reagente'], dtype=object)

In [92]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].value_counts()

NÃO REAGENTE     45
REAGENTE         34
Indeterminado     2
Name: DE_RESULTADO, dtype: int64

In [93]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
# Neste caso, vamos descartar o valor 'Indeterminado'
resultados_soro = ['REAGENTE', 'NÃO REAGENTE']
sel_soro_2 = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgG, Quimioluminescência') & (data_exames_HSL['DE_RESULTADO'].isin(resultados_soro))
exames_soro_2 = data_exames_HSL[sel_soro_2]

# Agora sim, mapeamos os resultados
resultados_map = {
   'REAGENTE' : 1, 
   'NÃO REAGENTE' : 0
}

exames_soro_2['DE_RESULTADO'] = exames_soro_2['DE_RESULTADO'].map(resultados_map)
exames_soro_2.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_2.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
91,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,1
1218,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,1
2856,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,0
2865,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,0
6021,F99249272E611207DBEC7EE3CF81F0E5,69ED1FBA20A1B0755943B89F82104ED8,2020-06-04,0
8958,74D237DB6D34A5BB2A95A26CD9FD5CF9,4FCD5F94A21B633D76E728EE17C4E46C,2020-06-27,0
18268,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,626B753CB67B893001127A521FED3960,2020-06-26,0
18276,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,6D5114B9017440527D233D0DAE085F1F,2020-07-09,1
19140,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,0
24204,C16902E84856371CFB33837851722042,66AF0EFC46FC52B87617914CE4780338,2020-06-03,0


#####**Covid 19, Anticorpos IgM, Quimioluminescência**

In [94]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgM, Quimioluminescência')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['Não reagente'], dtype=object)

In [95]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].value_counts()

NÃO REAGENTE     47
REAGENTE         25
Indeterminado     9
Name: DE_RESULTADO, dtype: int64

In [96]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
# Neste caso, vamos descartar o valor 'Indeterminado'
resultados_soro = ['REAGENTE', 'NÃO REAGENTE']
sel_soro_3 = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgM, Quimioluminescência') & (data_exames_HSL['DE_RESULTADO'].isin(resultados_soro))
exames_soro_3 = data_exames_HSL[sel_soro_3]

# Agora sim, mapeamos os resultados
resultados_map = {
   'REAGENTE' : 1, 
   'NÃO REAGENTE' : 0
}

exames_soro_3['DE_RESULTADO'] = exames_soro_3['DE_RESULTADO'].map(resultados_map)
exames_soro_3.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_3.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
93,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,1
1220,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,1
2867,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,0
6029,F99249272E611207DBEC7EE3CF81F0E5,69ED1FBA20A1B0755943B89F82104ED8,2020-06-04,0
8955,74D237DB6D34A5BB2A95A26CD9FD5CF9,4FCD5F94A21B633D76E728EE17C4E46C,2020-06-27,0
18272,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,626B753CB67B893001127A521FED3960,2020-06-26,0
18278,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,6D5114B9017440527D233D0DAE085F1F,2020-07-09,1
19142,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,0
24206,C16902E84856371CFB33837851722042,66AF0EFC46FC52B87617914CE4780338,2020-06-03,0
28250,36EC3ADD18AE59B3DC62C67B4A6440DA,907CDFDDDE3ECB18A2A705E91745FAEF,2020-06-08,0


#####**Covid 19, Anticorpos IgM, Quimiolumin.-Índice**

In [97]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgM, Quimiolumin.-Índice')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['inferior a 0,8', 'inferior a 0,7'], dtype=object)

In [98]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].value_counts()

0,6    19
0,5    17
0,7     9
0,8     5
1,7     3
0,9     3
1,3     2
1,5     2
1,4     2
1,8     2
1,6     2
2,1     1
1,2     1
0,3     1
1,0     1
0,4     1
2,7     1
4,3     1
2,2     1
9,2     1
2,5     1
5,6     1
4,1     1
2,0     1
6,5     1
3,1     1
Name: DE_RESULTADO, dtype: int64

In [99]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
# Neste caso, vamos considerar que valores inferiores a 0.7 como 0 e iguais
# ou superiores a 0.7 como 1
resultados_map = {
   '5,6' : 1, 
   '1,8' : 1, 
   '0,8' : 1, 
   '0,6' : 0, 
   '0,5' : 0, 
   '1,4' : 1, 
   '0,3' : 0, 
   '0,4' : 0, 
   '2,2' : 1,
   '1,6' : 1, 
   '1,7' : 1, 
   '4,3' : 1, 
   '1,2' : 1, 
   '2,5' : 1, 
   '2,1' : 1, 
   '0,7' : 1, 
   '1,3' : 1, 
   '0,9' : 1,
   '1,0' : 1, 
   '2,0' : 1, 
   '1,5' : 1, 
   '2,7' : 1, 
   '6,5' : 1, 
   '4,1' : 1, 
   '3,1' : 1, 
   '9,2' : 1
}

exames_soro_4 = data_exames_HSL[analito]
exames_soro_4['DE_RESULTADO'] = exames_soro_4['DE_RESULTADO'].map(resultados_map)
exames_soro_4.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_4.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
92,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,1
1219,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,1
2857,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,1
2866,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,0
6028,F99249272E611207DBEC7EE3CF81F0E5,69ED1FBA20A1B0755943B89F82104ED8,2020-06-04,0
8954,74D237DB6D34A5BB2A95A26CD9FD5CF9,4FCD5F94A21B633D76E728EE17C4E46C,2020-06-27,0
18271,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,626B753CB67B893001127A521FED3960,2020-06-26,0
18277,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,6D5114B9017440527D233D0DAE085F1F,2020-07-09,1
19141,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,0
24205,C16902E84856371CFB33837851722042,66AF0EFC46FC52B87617914CE4780338,2020-06-03,0


#####**Covid 19, Anticorpos Totais, Eletroquim-Índic**

In [100]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos Totais, Eletroquim-Índic')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['inferior a 0,6'], dtype=object)

In [101]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].unique()

array(['0,1', '144,0', '34,5', '45,7', '142,0', '25,3', '105,0', '6,5',
       '18,8', '13,8', '1,9', '0,2', '2,8', '6,6', '130,0', '5,6', '47,2',
       '55,4', '9,8', '79,2'], dtype=object)

In [102]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
# Neste caso, vamos considerar que valores inferiores a 0.6 como 0 e iguais
# ou superiores a 0.6 como 1
resultados_map = {
   '0,1' : 0, 
   '144,0' : 1, 
   '34,5' : 1, 
   '45,7' : 1, 
   '142,0' : 1, 
   '25,3' : 1, 
   '105,0' : 1, 
   '6,5' : 1,
   '18,8' : 1, 
   '13,8' : 1, 
   '1,9' : 1, 
   '0,2' : 0, 
   '2,8' : 1, 
   '6,6' : 1, 
   '130,0' : 1, 
   '5,6' : 1, 
   '47,2' : 1,
   '55,4' : 1, 
   '9,8' : 1, 
   '79,2' : 1
}

exames_soro_5 = data_exames_HSL[analito]
exames_soro_5['DE_RESULTADO'] = exames_soro_5['DE_RESULTADO'].map(resultados_map)
exames_soro_5.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_5.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
2853,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,0
35305,A5D8D706FE441C2060E2D180EC8709F0,8D09EF803B5BC8ED18F6F6947101803B,2020-07-14,1
99225,3DC7E7D2D0FD41DC69FDC7DE1C0D1165,121413D63C89B282947865446564E627,2020-07-23,1
99405,ED694041A831B3CF79B98D4659B03E99,BDF2314F34B5B1314FD1A0CF67957EB7,2020-07-30,1
120374,0375A54F4BAD09CEA28A86FEB1FD37AD,4695B23F40CBB5EE013290B1E149858C,2020-07-14,1
126588,2C0F9E3F2F4B1705DC919E2EEA1E72E0,F1113AC32BF8D25132075A2D182911F9,2020-07-13,1
150432,12A7373EDDEBBDC33A7A100602BE88F0,4B19E21886D2FA7AE5C3F0A220AEC9C4,2020-07-30,1
152294,0C398FB1D321F7159F93A84B3E426E48,31AEC40442113B4BA08DB1A405C6940B,2020-07-20,0
168365,BA790968FE3039D95F8B0EEE3A9C9B35,7A43189DE8542C8D2B69B47F7C405939,2020-07-21,0
168827,EFA28A2C29618475B2836000FE55EF69,4C784E823C0A2012D316A3116E6A2745,2020-07-31,1


#####**Covid 19, Anticorpos Totais, Eletroquimiolumi**

In [103]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos Totais, Eletroquimiolumi')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['Não reagente'], dtype=object)

In [104]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].value_counts()

REAGENTE        18
NÃO REAGENTE     6
Name: DE_RESULTADO, dtype: int64

In [105]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
resultados_map = {
   'REAGENTE' : 1, 
   'NÃO REAGENTE' : 0
}
exames_soro_6 = data_exames_HSL[analito]
exames_soro_6['DE_RESULTADO'] = exames_soro_6['DE_RESULTADO'].map(resultados_map)
exames_soro_6.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_6.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
2854,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,0
35306,A5D8D706FE441C2060E2D180EC8709F0,8D09EF803B5BC8ED18F6F6947101803B,2020-07-14,1
99226,3DC7E7D2D0FD41DC69FDC7DE1C0D1165,121413D63C89B282947865446564E627,2020-07-23,1
99406,ED694041A831B3CF79B98D4659B03E99,BDF2314F34B5B1314FD1A0CF67957EB7,2020-07-30,1
120375,0375A54F4BAD09CEA28A86FEB1FD37AD,4695B23F40CBB5EE013290B1E149858C,2020-07-14,1
126589,2C0F9E3F2F4B1705DC919E2EEA1E72E0,F1113AC32BF8D25132075A2D182911F9,2020-07-13,1
150433,12A7373EDDEBBDC33A7A100602BE88F0,4B19E21886D2FA7AE5C3F0A220AEC9C4,2020-07-30,1
152295,0C398FB1D321F7159F93A84B3E426E48,31AEC40442113B4BA08DB1A405C6940B,2020-07-20,0
168366,BA790968FE3039D95F8B0EEE3A9C9B35,7A43189DE8542C8D2B69B47F7C405939,2020-07-21,0
168828,EFA28A2C29618475B2836000FE55EF69,4C784E823C0A2012D316A3116E6A2745,2020-07-31,1


#####**Covid 19, Sorologia - Interpretação**

In [106]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Sorologia - Interpretação')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array([nan], dtype=object)

In [107]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].value_counts()

Detectados anticorpos da classe IgG contra SARS-CoV-2, Este perfil é compatível com infecção pregressa, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,                                                                                                                                                                                                                                                              8
Detectados anticorpos das classes IgM e IgG contra SARS-CoV-2, Este perfil sugere infecção recente, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos),

In [108]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
resultados_map = {
   'Ausência de anticorpos contra SARS-CoV-2 detectáveis nesta amostra, após análise em duas metodologias, Este perfil é compatível com ausência de infecção por este vírus, ou ausência de soroconversão no período decorrido entre a infecção e a coleta, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Desse modo, podem haver resultados negativos de sorologia mesmo em casos de doença anteriormente confirmada por PCR, Se persitir a hipótese diagnóstica, sugere-se seguimento sorológico em, no mínimo, 14 dias,' : 0,
   'Detectados anticorpos da classe IgG contra SARS-CoV-2, Este perfil é compatível com infecção pregressa, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,' : 1,
   'Detectados anticorpos das classes IgM e IgG contra SARS-CoV-2, Este perfil sugere infecção recente, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,' : 1,
   'Detectados anticorpos da classe IgM contra SARS-CoV-2, em apenas uma das metodologias utilizadas, A possibilidade de falsa reatividade não pode ser descartada, Sugere-se seguimento sorológico em, no mínimo, 7 dias,' : 1,
   'Detectados anticorpos da classe IgM contra SARS-CoV-2, Este perfil é compatível com soroconversão inicial ou produção de baixos níveis de anticorpos da classe IgG, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Sugere-se seguimento sorológico para avaliar a soroconversão de IgG em, no mínimo, 7 dias,' : 1
}
exames_soro_7 = data_exames_HSL[analito]
exames_soro_7['DE_RESULTADO'] = exames_soro_7['DE_RESULTADO'].map(resultados_map)
exames_soro_7.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_7.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
2859,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,0
35311,A5D8D706FE441C2060E2D180EC8709F0,8D09EF803B5BC8ED18F6F6947101803B,2020-07-14,1
99411,ED694041A831B3CF79B98D4659B03E99,BDF2314F34B5B1314FD1A0CF67957EB7,2020-07-30,1
120373,0375A54F4BAD09CEA28A86FEB1FD37AD,4695B23F40CBB5EE013290B1E149858C,2020-07-14,1
126594,2C0F9E3F2F4B1705DC919E2EEA1E72E0,F1113AC32BF8D25132075A2D182911F9,2020-07-13,1
150438,12A7373EDDEBBDC33A7A100602BE88F0,4B19E21886D2FA7AE5C3F0A220AEC9C4,2020-07-30,1
152291,0C398FB1D321F7159F93A84B3E426E48,31AEC40442113B4BA08DB1A405C6940B,2020-07-20,0
168371,BA790968FE3039D95F8B0EEE3A9C9B35,7A43189DE8542C8D2B69B47F7C405939,2020-07-21,1
168833,EFA28A2C29618475B2836000FE55EF69,4C784E823C0A2012D316A3116E6A2745,2020-07-31,1
169110,C33DEE8231AD0374CAF93E52C5A473E5,487702DD590C0689BDC7B253E37538D4,2020-08-03,1


#####**Covid 19, Sorologia - Conclusão**

In [109]:
# Temos resultados tanto numéricos quanto categóricos, que descrevem se foi detectado COVID ou não no paciente.
# Nesse caso, precisamos analisar os analitos separadamente, caracterizando para cada um se o paciente teve ou
# não COVID. Essa análise mais detalhada é importante, pois a princípio, os testes sorológicos são mais confiáveis
# do que os testes PCR e os testes rápidos.
data_exames_HSL[exames_covid_soro]['DE_ANALITO'].unique()

array(['Covid 19, Anticorpos IgG, Quimiolumin.-Índice',
       'Covid 19, Anticorpos IgG, Quimioluminescência',
       'Covid 19, Anticorpos IgM, Quimiolumin.-Índice',
       'Covid 19, Anticorpos IgM, Quimioluminescência',
       'Covid 19, Anticorpos Totais, Eletroquim-Índic',
       'Covid 19, Anticorpos Totais, Eletroquimiolumi',
       'Covid 19, Sorologia - Interpretação',
       'Covid 19, Sorologia - Conclusão'], dtype=object)

In [110]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Sorologia - Conclusão')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array([nan], dtype=object)

In [111]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].unique()

array(['Amostra REAGENTE para IgM e IgG contra SARS-CoV-2,',
       'Amostra REAGENTE para IgG contra SARS-CoV-2,',
       'Amostra REAGENTE para IgM contra SARS-CoV-2,'], dtype=object)

In [112]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
resultados_map = {
   'Amostra REAGENTE para IgM e IgG contra SARS-CoV-2,' : 1,
   'Amostra REAGENTE para IgG contra SARS-CoV-2,' : 1,
   'Amostra REAGENTE para IgM contra SARS-CoV-2,' : 1
}
exames_soro_8 = data_exames_HSL[analito]
exames_soro_8['DE_RESULTADO'] = exames_soro_8['DE_RESULTADO'].map(resultados_map)
exames_soro_8.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_8.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
99412,ED694041A831B3CF79B98D4659B03E99,BDF2314F34B5B1314FD1A0CF67957EB7,2020-07-30,1
150439,12A7373EDDEBBDC33A7A100602BE88F0,4B19E21886D2FA7AE5C3F0A220AEC9C4,2020-07-30,1
168834,EFA28A2C29618475B2836000FE55EF69,4C784E823C0A2012D316A3116E6A2745,2020-07-31,1
169111,C33DEE8231AD0374CAF93E52C5A473E5,487702DD590C0689BDC7B253E37538D4,2020-08-03,1
196147,272EA87C1ECF3B38BE4D127B4B546408,B91F954CD4BAE535AAF5CE3B0D849EC6,2020-08-05,1
202352,5983EE3C052F98669F571E7235C43E05,7C5773B4522179D2AC5078F0F3EC9EF9,2020-08-02,1
232864,64749E97F6737AE1B293B8B4A23A0C16,AD28C524F5882EC5E351BCD54101A9A4,2020-08-03,1
242607,F9B7D5131D98174A6951210AAB4ECB7E,1E97DF294207A4FBDC373B995BB79A35,2020-08-10,1


In [113]:
# Precisamos descobrir qual o valor de referência
sel = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro')
data_exames_HSL[sel]['DE_VALOR_REFERENCIA'].unique()

array(['inferior a 0,8', 'Não reagente', 'inferior a 0,6', nan,
       'inferior a 0,9', 'inferior a 0,7'], dtype=object)

In [114]:
# Dos resultados distintos observados na lista acima, nos interessamos somente pelos que permitem identificar
# ou não a presença de anticorpos para COVID-19 nos pacientes, representados na lista abaixo.
resultados_soro = ['REAGENTE', 
                   'NÃO REAGENTE',  
                   'Indeterminado',
                   'Ausência de anticorpos contra SARS-CoV-2 detectáveis nesta amostra, após análise em duas metodologias, Este perfil é compatível com ausência de infecção por este vírus, ou ausência de soroconversão no período decorrido entre a infecção e a coleta, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Desse modo, podem haver resultados negativos de sorologia mesmo em casos de doença anteriormente confirmada por PCR, Se persitir a hipótese diagnóstica, sugere-se seguimento sorológico em, no mínimo, 14 dias,', 'Detectados anticorpos da classe IgG contra SARS-CoV-2, Este perfil é compatível com infecção pregressa, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,',
                   'Detectados anticorpos das classes IgM e IgG contra SARS-CoV-2, Este perfil sugere infecção recente, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,',
                   'Amostra REAGENTE para IgM e IgG contra SARS-CoV-2,', 
                   'Amostra REAGENTE para IgG contra SARS-CoV-2,', 
                   'Detectados anticorpos da classe IgM contra SARS-CoV-2, em apenas uma das metodologias utilizadas, A possibilidade de falsa reatividade não pode ser descartada, Sugere-se seguimento sorológico em, no mínimo, 7 dias,',
                   'Detectados anticorpos da classe IgM contra SARS-CoV-2, Este perfil é compatível com soroconversão inicial ou produção de baixos níveis de anticorpos da classe IgG, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Sugere-se seguimento sorológico para avaliar a soroconversão de IgG em, no mínimo, 7 dias,',
                   'Amostra REAGENTE para IgM contra SARS-CoV-2,']

In [115]:
# Selecionando os testes sorológicos com somente os resultados listados acima
sel_exames_soro = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro') & (data_exames_HSL[exames_covid_soro]['DE_RESULTADO'].isin(resultados_soro))
exames_soro = data_exames_HSL[sel_exames_soro]
exames_soro.head(10)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
91,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,Recepção do Centro Diagnóstico,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",REAGENTE,NaN,Não reagente
93,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,Recepção do Centro Diagnóstico,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",REAGENTE,NaN,Não reagente
1218,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",REAGENTE,NaN,Não reagente
1220,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",REAGENTE,NaN,Não reagente
2854,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos Totais, Eletroquimiolumi",NÃO REAGENTE,NaN,Não reagente
2856,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",NÃO REAGENTE,UA/mL,Não reagente
2858,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",Indeterminado,UA/mL,Não reagente
2859,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Sorologia - Interpretação",Ausência de anticorpos contra SARS-CoV-2 detec...,NaN,NaN
2865,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",NÃO REAGENTE,NaN,Não reagente
2867,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",NÃO REAGENTE,NaN,Não reagente


In [116]:
# Precisamos agora mapear cada um desses resultados para detecção ou não de COVID-19:
#
# 0 => Não foram detectados anticorpos para COVID-19
# 1 => Detectados anticorpos para COVID-19
soro_map = {
    'REAGENTE': 1,
    'NÃO REAGENTE': 0,
    'Indeterminado' : 0,
    'Ausência de anticorpos contra SARS-CoV-2 detectáveis nesta amostra, após análise em duas metodologias, Este perfil é compatível com ausência de infecção por este vírus, ou ausência de soroconversão no período decorrido entre a infecção e a coleta, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Desse modo, podem haver resultados negativos de sorologia mesmo em casos de doença anteriormente confirmada por PCR, Se persitir a hipótese diagnóstica, sugere-se seguimento sorológico em, no mínimo, 14 dias,' : 0,
    'Detectados anticorpos das classes IgM e IgG contra SARS-CoV-2, Este perfil sugere infecção recente, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,' : 1,
    'Amostra REAGENTE para IgM e IgG contra SARS-CoV-2,': 1,
    'Amostra REAGENTE para IgG contra SARS-CoV-2,' : 1, 
    'Detectados anticorpos da classe IgM contra SARS-CoV-2, em apenas uma das metodologias utilizadas, A possibilidade de falsa reatividade não pode ser descartada, Sugere-se seguimento sorológico em, no mínimo, 7 dias,' : 1,                   
    'Detectados anticorpos da classe IgM contra SARS-CoV-2, Este perfil é compatível com soroconversão inicial ou produção de baixos níveis de anticorpos da classe IgG, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Sugere-se seguimento sorológico para avaliar a soroconversão de IgG em, no mínimo, 7 dias,' : 0,
    'Amostra REAGENTE para IgM contra SARS-CoV-2,' : 1
}

exames_soro['DE_RESULTADO'] = exames_soro['DE_RESULTADO'].map(soro_map)
exames_soro.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
91,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,Recepção do Centro Diagnóstico,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",1.0,NaN,Não reagente
93,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,Recepção do Centro Diagnóstico,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",1.0,NaN,Não reagente
1218,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",1.0,NaN,Não reagente
1220,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-06-07,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",1.0,NaN,Não reagente
2854,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos Totais, Eletroquimiolumi",0.0,NaN,Não reagente
2856,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",0.0,UA/mL,Não reagente
2858,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",0.0,UA/mL,Não reagente
2859,C3A3D083DBC1353FAB5F87282EE86633,6A9B541EE6A6510F2803E96E47EEB5DA,2020-07-23,Laboratório de Patologia Clínica,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Sorologia - Interpretação",0.0,NaN,NaN
2865,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgG, Quimioluminescência",0.0,NaN,Não reagente
2867,BEBA451E2C91A4D869D14EE790FE78FD,F00A8B071066A41020B80A1F2F590663,2020-05-31,UTI,COVID-19-Sorologia IgM e IgG por quimiluminesc...,"Covid 19, Anticorpos IgM, Quimioluminescência",0.0,NaN,Não reagente


In [117]:
# Agora analsamos quantos resultados de testes sorológicos ainda temos:
print('Após tratar os dados, temos {} exames sorológicos realizados'.format(exames_soro.shape[0]))

Após tratar os dados, temos 216 exames sorológicos realizados


####**3.17.2) COVID-19-Sorologia IgG**

In [118]:
# Resultados dos exames de sorologia IgG
data_exames_HSL[exames_corona_igg]['DE_RESULTADO'].unique()

array(['NÃO REAGENTE', '0,14', 'REAGENTE', '1,28', '0,56', '5,26', '0,69',
       '11,01', '10,83', '1,36', '2,73', '5,27', '0,22', '5,41', '12,61',
       '7,70', '8,67', '5,17', '12,16', '0,24', '0,23', '12,10', '4,84',
       '0,28', '0,31', '0,19', '0,17', '6,17', '0,16', '0,33', '15,09',
       '5,45', '1,96', '0,21', '15,26', '0,39', '2,48', '0,30', '0,27',
       '0,36', '0,12'], dtype=object)

In [119]:
# Vamos analisar quais os resultados de testes que temos no conjunto de dados, 
# quando analisamos os testes sorológicos para COVID-19
data_exames_HSL[exames_corona_igg]['DE_ANALITO'].unique()

array(['Covid 19, Anticorpos IgG', 'Covid 19, Anticorpos IgG, índice'],
      dtype=object)

#####**Covid 19, Anticorpos IgG**

In [120]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'Sorologia - Coronavírus, IgG') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgG')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array(['Não reagente'], dtype=object)

In [121]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].unique()

array(['NÃO REAGENTE', 'REAGENTE'], dtype=object)

In [122]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
resultados_map = {
   'NÃO REAGENTE' : 0, 
   'REAGENTE' : 1
}

exames_soro_9 = data_exames_HSL[analito]
exames_soro_9['DE_RESULTADO'] = exames_soro_9['DE_RESULTADO'].map(resultados_map)
exames_soro_9.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_9.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
6238,FE1BAA57FC75EF9A274F91381387B493,9DBBD1A65AEDE736D80AE59D895A08C3,2020-04-04,0
18689,47FC4D62E7F52E4E4B346B5915863384,B3A6E2E8822314A6057A1616939472CA,2020-05-11,1
27999,36EC3ADD18AE59B3DC62C67B4A6440DA,0165A9BBADB9C245FDE6F566D5BB1954,2020-05-12,0
54819,90ADCB85ADEAD09537C99E74E115D68E,F8DF3DC4EA3642BAE675BAE80A828065,2020-05-13,1
60716,A7CFB63755E5541A74CA7E12C7575A48,FFEBC1327D2CF4CCA5307D4D2009A30E,2020-05-08,0
61440,5375632434842C97BD3BD355C4CBFD91,A284AA6A3B56E5381D37B1D0627D1939,2020-05-20,1
74413,1367B8AD3AA4DB78C3D078361096B847,F4748BDDBDEED6A716503BC78F349BE7,2020-04-29,1
75269,AC54FA57046CFA8EA0B4C0913D48A00C,7CFA6E2534B02BF49BCA21DB4930DC62,2020-05-12,1
82080,4AF35DA21A9FC69AD115FF35A94C1391,F87C7EB3940FA74B81CB1C5AAF8941E2,2020-04-16,1
95415,3834EE34C6B4325ACAEBD1FA03921BA4,C01326E59ADADE14A73589C1A0AF2264,2020-04-20,1


#####**Covid 19, Anticorpos IgG, índice**

In [123]:
# Para cada analito, precisamos saber o valor de referência para identificar se os resultados indicam ou não
# presença de COVID.
analito = (data_exames_HSL['DE_EXAME'] == 'Sorologia - Coronavírus, IgG') & (data_exames_HSL['DE_ANALITO'] == 'Covid 19, Anticorpos IgG, índice')
data_exames_HSL[analito]['DE_VALOR_REFERENCIA'].unique()

array([nan, 'inferior a 0,80'], dtype=object)

In [124]:
# Obtendo os valores numéricos referentes ao analito atual
data_exames_HSL[analito]['DE_RESULTADO'].value_counts()

0,23     2
0,17     2
0,14     2
0,56     2
0,31     1
0,69     1
0,21     1
1,96     1
8,67     1
1,28     1
0,12     1
2,48     1
5,27     1
5,41     1
1,36     1
6,17     1
12,61    1
0,30     1
10,83    1
0,39     1
0,27     1
0,16     1
7,70     1
15,09    1
2,73     1
0,36     1
15,26    1
12,16    1
11,01    1
5,26     1
0,19     1
0,22     1
0,33     1
5,45     1
12,10    1
0,24     1
4,84     1
5,17     1
0,28     1
Name: DE_RESULTADO, dtype: int64

In [125]:
# Vamos adotar como padrão que:
#
# 0 => COVID não detectado no paciente
# 1 => COVID detectado no paciente
#
# Consideramos valores inferiores a 0.8 como 0
resultados_map = {
   '0,14' : 0, 
   '1,28' : 1, 
   '0,56' : 0, 
   '5,26' : 1, 
   '0,69' : 0, 
   '11,01' : 1, 
   '10,83' : 1, 
   '1,36' : 1,
   '2,73' : 1, 
   '5,27' : 1, 
   '0,22' : 0, 
   '5,41' : 1, 
   '12,61' : 1, 
   '7,70' : 1, 
   '8,67' : 1, 
   '5,17' : 1,
   '12,16' : 1, 
   '0,24' : 0, 
   '0,23' : 0, 
   '12,10' : 1, 
   '4,84' : 1, 
   '0,28' : 0, 
   '0,31' : 0, 
   '0,19' : 0,
   '0,17' : 0, 
   '6,17' : 1, 
   '0,16' : 0, 
   '0,33' : 0, 
   '15,09' : 1, 
   '5,45' : 1, 
   '1,96' : 1, 
   '0,21' : 0,
   '15,26' : 1, 
   '0,39' : 0, 
   '2,48' : 1, 
   '0,30' : 0, 
   '0,27' : 0, 
   '0,36' : 0, 
   '0,12' : 0
}

exames_soro_10 = data_exames_HSL[analito]
exames_soro_10['DE_RESULTADO'] = exames_soro_10['DE_RESULTADO'].map(resultados_map)
exames_soro_10.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_10.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
6239,FE1BAA57FC75EF9A274F91381387B493,9DBBD1A65AEDE736D80AE59D895A08C3,2020-04-04,0
18690,47FC4D62E7F52E4E4B346B5915863384,B3A6E2E8822314A6057A1616939472CA,2020-05-11,1
28000,36EC3ADD18AE59B3DC62C67B4A6440DA,0165A9BBADB9C245FDE6F566D5BB1954,2020-05-12,0
54820,90ADCB85ADEAD09537C99E74E115D68E,F8DF3DC4EA3642BAE675BAE80A828065,2020-05-13,1
60717,A7CFB63755E5541A74CA7E12C7575A48,FFEBC1327D2CF4CCA5307D4D2009A30E,2020-05-08,0
61441,5375632434842C97BD3BD355C4CBFD91,A284AA6A3B56E5381D37B1D0627D1939,2020-05-20,1
74414,1367B8AD3AA4DB78C3D078361096B847,F4748BDDBDEED6A716503BC78F349BE7,2020-04-29,1
75270,AC54FA57046CFA8EA0B4C0913D48A00C,7CFA6E2534B02BF49BCA21DB4930DC62,2020-05-12,1
82081,4AF35DA21A9FC69AD115FF35A94C1391,F87C7EB3940FA74B81CB1C5AAF8941E2,2020-04-16,1
95416,3834EE34C6B4325ACAEBD1FA03921BA4,C01326E59ADADE14A73589C1A0AF2264,2020-04-20,1


#### **3.17.3) COVID-19-Sorologia IgA**

In [126]:
# Resultados dos exames de sorologia IgA
data_exames_HSL[exames_corona_iga]['DE_RESULTADO'].unique()

array(['REAGENTE', '4,66', '2,75', 'NÃO REAGENTE', '0,22', '7,48',
       '33,89', '1,41', '1,83', '4,92', '5,90', '36,91', '6,61', '8,82',
       '5,49', '0,20', '3,27', '5,98', '0,45', '5,97', '0,34', '0,38',
       '4,17', '5,72', '0,54', '0,52', '42,32', '0,43', '4,74', '0,73',
       '0,16', '0,15', '0,21', '0,36'], dtype=object)

In [127]:
# Vamos analisar os analitos para o exame sorológico de IgA
data_exames_HSL[exames_corona_iga]['DE_ANALITO'].unique()

array(['Covid 19, Anticorpos IgA', 'Covid 19, Anticorpos IgA, índice'],
      dtype=object)

In [128]:
# Pelos exemplos anteriores podemos entender que os valores numéricos referem-se ao analito índice
# e os valores REAGENTE e NAO REAGENTE à presença de anticorpos IgA. Vamos verificar os valores
# de referência
data_exames_HSL[exames_corona_iga]['DE_VALOR_REFERENCIA'].unique()

array(['Não reagente', 'inferior a 0,80', nan], dtype=object)

In [129]:
# Ou seja, mantendo o padrão anterior:
#
# 0 => Não detecção de anticorpos para COVID
# 1 => Detecção de anticorpos para COVID
#
# Consideramos REAGENTE = 1, NAO REAGENTE = 1 e valores numéricos inferiores a 0.8 = 0:
resultados_map = {
   'REAGENTE' : 1, 
   '4,66' : 1, 
   '2,75' : 1, 
   'NÃO REAGENTE' : 0, 
   '0,22' : 0, 
   '7,48' : 1,
   '33,89' : 1, 
   '1,41' : 1, 
   '1,83' : 1, 
   '4,92' : 1, 
   '5,90' : 1, 
   '36,91' : 1, 
   '6,61' : 1, 
   '8,82' : 1,
   '5,49' : 1, 
   '0,20' : 0, 
   '3,27' : 1, 
   '5,98' : 1, 
   '0,45' : 0, 
   '5,97' : 1, 
   '0,34' : 0, 
   '0,38' : 0,
   '4,17' : 1, 
   '5,72' : 1, 
   '0,54' : 0, 
   '0,52' : 0, 
   '42,32' : 1, 
   '0,43' : 0, 
   '4,74' : 1, 
   '0,73' : 0,
   '0,16' : 0, 
   '0,15' : 0, 
   '0,21' : 0, 
   '0,36' : 0
}

exames_soro_11 = data_exames_HSL[exames_corona_iga]
exames_soro_11['DE_RESULTADO'] = exames_soro_11['DE_RESULTADO'].map(resultados_map)
exames_soro_11.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_soro_11.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
18691,47FC4D62E7F52E4E4B346B5915863384,B3A6E2E8822314A6057A1616939472CA,2020-05-11,1
18692,47FC4D62E7F52E4E4B346B5915863384,B3A6E2E8822314A6057A1616939472CA,2020-05-11,1
54821,90ADCB85ADEAD09537C99E74E115D68E,F8DF3DC4EA3642BAE675BAE80A828065,2020-05-13,1
54822,90ADCB85ADEAD09537C99E74E115D68E,F8DF3DC4EA3642BAE675BAE80A828065,2020-05-13,1
60714,A7CFB63755E5541A74CA7E12C7575A48,FFEBC1327D2CF4CCA5307D4D2009A30E,2020-05-08,0
60715,A7CFB63755E5541A74CA7E12C7575A48,FFEBC1327D2CF4CCA5307D4D2009A30E,2020-05-08,0
61438,5375632434842C97BD3BD355C4CBFD91,A284AA6A3B56E5381D37B1D0627D1939,2020-05-20,1
61439,5375632434842C97BD3BD355C4CBFD91,A284AA6A3B56E5381D37B1D0627D1939,2020-05-20,1
74411,1367B8AD3AA4DB78C3D078361096B847,F4748BDDBDEED6A716503BC78F349BE7,2020-04-29,1
74412,1367B8AD3AA4DB78C3D078361096B847,F4748BDDBDEED6A716503BC78F349BE7,2020-04-29,1


####**3.17.4) Combinando dados de exames sorológicos**

In [130]:
# Agora que analisamos todos os resultados separadamente dos exames sorológicos, podemos unir todos eles em
# um único dataframe
concat_soro = [exames_soro_1, exames_soro_2, exames_soro_3, exames_soro_4, exames_soro_5, exames_soro_6, exames_soro_7, exames_soro_8, exames_soro_9, exames_soro_10, exames_soro_11]
exames_soro_total = pd.concat(concat_soro, axis=0, ignore_index=True)
exames_soro_total.tail(10)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
533,1588840B45EC66349AB51CD201EC2276,EF88494900F114B86F7E37A14701BB57,2020-05-11,0
534,1588840B45EC66349AB51CD201EC2276,EF88494900F114B86F7E37A14701BB57,2020-05-11,0
535,E1DB75E3106A496D885700D3A372486D,122FC96433E8054979AEBAA66FDF50B6,2020-04-30,0
536,E1DB75E3106A496D885700D3A372486D,122FC96433E8054979AEBAA66FDF50B6,2020-04-30,0
537,E1DB75E3106A496D885700D3A372486D,122FC96433E8054979AEBAA66FDF50B6,2020-04-27,0
538,E1DB75E3106A496D885700D3A372486D,122FC96433E8054979AEBAA66FDF50B6,2020-04-27,0
539,E1DB75E3106A496D885700D3A372486D,122FC96433E8054979AEBAA66FDF50B6,2020-05-07,0
540,E1DB75E3106A496D885700D3A372486D,122FC96433E8054979AEBAA66FDF50B6,2020-05-07,0
541,9155AA73E75993CA3038A3C018C759E6,2B4D126DF82BCB7C874DD817F2DB92D9,2020-04-25,0
542,9155AA73E75993CA3038A3C018C759E6,2B4D126DF82BCB7C874DD817F2DB92D9,2020-04-25,0


In [131]:
# Verificando que não temos entradas não nulas
exames_soro_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID_PACIENTE     543 non-null    object
 1   ID_ATENDIMENTO  543 non-null    object
 2   DT_COLETA       543 non-null    object
 3   DE_RESULTADO    543 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 17.1+ KB


### **3.18) Análise dos testes PCR**

##### **3.18.1) COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)**

In [132]:
# Temos dois tipos de testes PCR para analisar.
# Vamos começar analisando os resultados do exame 'COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)'
data_exames_HSL[exames_covid_pcr]['DE_RESULTADO'].value_counts()

swab de nasofaringe                           1249
DETECTADO (POSITIVO)                           621
DETECTADO                                      600
NÃO DETECTADO                                  210
raspado de nasofaringe                          92
NÃO DETECTADO (NEGATIVO)                        91
raspado de material do trato respiratorio       62
DETECTÁVEL                                      32
secrecao traqueal                               30
raspado de material do trato respiratório       19
aspirado de nasofaringe                         16
lavado de nasofaringe                           14
raspado de orofaringe e nasofaringe              8
INDETECTÁVEL                                     6
aspirado de material do trato respiratorio       6
SWAB NASOFARINGEO                                2
secreção traqueal                                2
INCONCLUSIVO                                     2
Inconclusivo                                     1
lavado de material do trato res

In [133]:
# Dos resultados distintos observados na lista acima, nos interessamos somente pelos que permitem identificar
# ou não a presença de anticorpos para COVID-19 nos pacientes, representados na lista abaixo.
resultados_PCR = ['DETECTADO', 'DETECTADO (POSITIVO)',
       'INDETECTÁVEL', 'NÃO DETECTADO (NEGATIVO)',
       'DETECTÁVEL',
       'NÃO DETECTADO']

# Selecionando somente os resultados que interessam
sel_exames_PCR = (data_exames_HSL['DE_EXAME'] == 'COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)') & (data_exames_HSL[exames_covid_pcr]['DE_RESULTADO'].isin(resultados_PCR))
exames_PCR_1 = data_exames_HSL[sel_exames_PCR]
exames_PCR_1.head(10)       

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
149,0AD1FFA4419472256666A3445414F1F9,70A834FEFF3562FCBA3888D592DA2609,2020-03-16,Recepção do Centro Diagnóstico,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO,NaN,Não detectado
348,FD6F18D3E9DA3ADD5179E6B3EA21ABB2,68623C98C8B0B6AA799F1CCE5A8033CA,2020-06-22,Recepção do Centro Diagnóstico,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO (POSITIVO),NaN,Não detectado
351,3B9836D0E92704BFAB0B2E691D0A3BAE,F6A97EB5859A622FB6EB9170E677CEB4,2020-03-12,Programa Cuidando de Quem Cuida,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),INDETECTÁVEL,NaN,Indetectável
353,3B9836D0E92704BFAB0B2E691D0A3BAE,A5ABEF18B34146C64FF12F2382ACA303,2020-06-29,Programa Cuidando de Quem Cuida,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO (POSITIVO),NaN,Não detectado
355,C40D5F4CFDAA517A9513147B03BB82B4,F30D42CC704E8D4E5F3D7A9F279E8E4F,2020-07-13,Programa Cuidando de Quem Cuida,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO (POSITIVO),NaN,Não detectado
357,B38CC72A753900257E4D485795B106FD,73CEC46EE6DB4CDEC0371611BF9D557F,2020-03-17,Recepção do Centro Diagnóstico,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO,NaN,Não detectado
428,D1C4E29D126CA8B4359827903CB222AC,F199DE2360D5172CC936FB926344F021,2020-07-14,Pronto Socorro,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO (POSITIVO),NaN,Não detectado
536,EA3D530A51086CEFC8C6DFAEC53D00F2,E924C3167E6FB451D00DFAE361B6B261,2020-03-27,Unidades de Internação,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO,NaN,Não detectado
2048,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-05-21,UTI,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),DETECTADO,NaN,Não detectado
2750,C3A3D083DBC1353FAB5F87282EE86633,986F5A4D204635B958B17EB882063D9C,2020-03-14,Pronto Socorro,"COVID-19-PCR para SARS-COV-2, Vários Materiais...",Coronavírus (2019-nCoV),INDETECTÁVEL,NaN,DETECTÁVEL


In [134]:
# Precisamos agora mapear cada um desses resultados para detecção ou não de COVID-19:
#
# 0 => Não foram detectados anticorpos para COVID-19
# 1 => Detectados anticorpos para COVID-19
PCR_map = {
    'DETECTADO' : 1, 
    'DETECTADO (POSITIVO)' : 1,
    'INDETECTÁVEL' : 0, 
    'NÃO DETECTADO (NEGATIVO)' : 0,
    'DETECTÁVEL' : 1,
    'NÃO DETECTADO' : 0,
    'Inconclusivo ' : 0,
    'Inconclusivo' : 0
}

# Mapeando os novos valores
exames_PCR_1['DE_RESULTADO'] = exames_PCR_1['DE_RESULTADO'].map(PCR_map)
exames_PCR_1.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_PCR_1.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
149,0AD1FFA4419472256666A3445414F1F9,70A834FEFF3562FCBA3888D592DA2609,2020-03-16,1
348,FD6F18D3E9DA3ADD5179E6B3EA21ABB2,68623C98C8B0B6AA799F1CCE5A8033CA,2020-06-22,1
351,3B9836D0E92704BFAB0B2E691D0A3BAE,F6A97EB5859A622FB6EB9170E677CEB4,2020-03-12,0
353,3B9836D0E92704BFAB0B2E691D0A3BAE,A5ABEF18B34146C64FF12F2382ACA303,2020-06-29,1
355,C40D5F4CFDAA517A9513147B03BB82B4,F30D42CC704E8D4E5F3D7A9F279E8E4F,2020-07-13,1
357,B38CC72A753900257E4D485795B106FD,73CEC46EE6DB4CDEC0371611BF9D557F,2020-03-17,1
428,D1C4E29D126CA8B4359827903CB222AC,F199DE2360D5172CC936FB926344F021,2020-07-14,1
536,EA3D530A51086CEFC8C6DFAEC53D00F2,E924C3167E6FB451D00DFAE361B6B261,2020-03-27,1
2048,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-05-21,1
2750,C3A3D083DBC1353FAB5F87282EE86633,986F5A4D204635B958B17EB882063D9C,2020-03-14,0


In [135]:
# Agora analsamos quantos resultados de testes sorológicos ainda temos:
print('Após tratar os dados, temos {} exames PCR realizados'.format(exames_PCR_1.shape[0]))

Após tratar os dados, temos 1560 exames PCR realizados


#####**3.18.2) Detecção de Coronavírus (NCoV-2019) POR PCR (Anatomia Patológica)**

In [136]:
# Agora vamos analisar os resultados do segundo tipo de exame PCR realizado.
data_exames_HSL[exames_corona_pcr]['DE_RESULTADO'].unique()

array(['DETECTADO'], dtype=object)

In [137]:
# Ou seja, todas as pessoas que se submeteram a esse exame testaram positivo para COVID-19.
# Assim, podemos tratar esses dados diretamente mapeando os resultados para 1:
exames_PCR_2 = data_exames_HSL[exames_corona_pcr]
exames_PCR_2['DE_RESULTADO'] = 1
exames_PCR_2.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_PCR_2.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
150,3487791F44C34B421C932DC8616A8437,58DB2576E7220A442BC49D2F02832708,2020-05-04,1
396,3EB01D5BF412F46BE68D184C7981D84F,842CB44AA0F60D2F6FA8A175853CE9AC,2020-05-03,1
397,B9697B4994DF6367EDB722F1EC4AB423,0E944DFEEFAACAEDAABAC8DCB945608C,2020-05-18,1
2749,4EC36E0CC80E9456E653791F68423A9E,16F6DB22F8934073DE7C2E6E5DA37570,2020-07-06,1
3068,2A17CD3A45ED87C2F638AFB4917B463B,7B65766B31FFF3808D40BFE48A5050AD,2020-06-06,1
3179,85D46882DBFFD6680B2BBE87FFCDBFE1,A10AF51A7E4ED37FFE3F25CC90D0CDF2,2020-08-08,1
3424,2D6E2EFC360FA4829A61AE07EBAEBBAF,AB86038D2E008C76283885A95E81E5C1,2020-06-17,1
3707,6960A46A81F18DECDF6CC84FC6C568B8,007D712BAD4C4B8DCCCE9068A3968B12,2020-05-09,1
3814,88D972DF3C4F6E8560CE119ED5445BDA,1C3A12045AE1D7F78B12095C47B7E922,2020-08-04,1
3820,4CE0DB6566E705FE834E650C1C1DFE3D,2A6294EA16DD2CE08B86FC7BF3857E1F,2020-05-30,1


In [138]:
# Agora analsamos quantos resultados de testes sorológicos ainda temos:
print('Após tratar os dados, temos {} exames PCR realizados'.format(exames_PCR_2.shape[0]))

Após tratar os dados, temos 492 exames PCR realizados


#####**3.18.3) Combinando os testes PCR**

In [139]:
# Vamos unir todos os dados de exames PCR (exames_PCR_1 + exames_PCR_2)
exames_PCR_total = pd.concat([exames_PCR_1, exames_PCR_2], axis=0, ignore_index=True)
exames_PCR_total.head(10)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
0,0AD1FFA4419472256666A3445414F1F9,70A834FEFF3562FCBA3888D592DA2609,2020-03-16,1
1,FD6F18D3E9DA3ADD5179E6B3EA21ABB2,68623C98C8B0B6AA799F1CCE5A8033CA,2020-06-22,1
2,3B9836D0E92704BFAB0B2E691D0A3BAE,F6A97EB5859A622FB6EB9170E677CEB4,2020-03-12,0
3,3B9836D0E92704BFAB0B2E691D0A3BAE,A5ABEF18B34146C64FF12F2382ACA303,2020-06-29,1
4,C40D5F4CFDAA517A9513147B03BB82B4,F30D42CC704E8D4E5F3D7A9F279E8E4F,2020-07-13,1
5,B38CC72A753900257E4D485795B106FD,73CEC46EE6DB4CDEC0371611BF9D557F,2020-03-17,1
6,D1C4E29D126CA8B4359827903CB222AC,F199DE2360D5172CC936FB926344F021,2020-07-14,1
7,EA3D530A51086CEFC8C6DFAEC53D00F2,E924C3167E6FB451D00DFAE361B6B261,2020-03-27,1
8,F5B5BD59D98836D8AA2D8C27B01225AB,FBF3FC0FD4890AF6B6EBFE6BFC12EC0C,2020-05-21,1
9,C3A3D083DBC1353FAB5F87282EE86633,986F5A4D204635B958B17EB882063D9C,2020-03-14,0


### **3.19) Análise dos testes rápidos**

In [140]:
data_exames_HSL[exames_covid_rapido]['DE_ANALITO'].unique()

array(['Covid 19, Material', 'Covid 19, Anticorpos IgM, teste rápido',
       'Covid 19, Anticorpos IgG, teste rápido',
       'Covid 19,Antic IgM/IgG,teste rápido-Interpret',
       'Covid 19, Antic IgM/IgG, teste rápido-Observ'], dtype=object)

In [141]:
# Faremos a mesma análise para os testes rápidos. Analisamos primeiro os tipos de resultados encontrados para esse teste
data_exames_HSL[exames_covid_rapido]['DE_RESULTADO'].unique()

array(['plasma', 'NÃO REAGENTE', 'REAGENTE',
       'Evidência sorológica de infecção pregressa por SARS-CoV-2,',
       'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, o momento da soroconversão pode ser variável de indivíduo para indivíduo, podendo ser afetado pelo quadro clínico apresentado, O resultado sugere que já tenham transcorrido mais de 3 semanas desde a infecção aguda, A capacidade protetora dos anticorpos da classe IgG não é completamente estabelecida,;',
       'Sem evidência sorológica de infecção por SARS-CoV-2,',
       'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, pode não haver anticorpos detectáveis (resultado negativo), inclusive em pessoas que tiveram infecção previamente confirmada por PCR, Se persis

In [142]:
# Dos resultados distintos observados na lista acima, nos interessamos somente pelos que permitem identificar
# ou não a presença de anticorpos para COVID-19 nos pacientes, representados na lista abaixo.
resultados_rapido = ['NÃO REAGENTE', 'REAGENTE',
       'Evidência sorológica de infecção pregressa por SARS-CoV-2,',
       'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, o momento da soroconversão pode ser variável de indivíduo para indivíduo, podendo ser afetado pelo quadro clínico apresentado, O resultado sugere que já tenham transcorrido mais de 3 semanas desde a infecção aguda, A capacidade protetora dos anticorpos da classe IgG não é completamente estabelecida,;',
       'Sem evidência sorológica de infecção por SARS-CoV-2,',
       'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, pode não haver anticorpos detectáveis (resultado negativo), inclusive em pessoas que tiveram infecção previamente confirmada por PCR, Se persistir a hipótese diagnóstica, sugere-se, a critério médico, repetir o teste em nova amostra colhida em 7 dias,;',
       'Evidência sorológica de infecção recente por SARS-CoV-2,',
       'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, o momento da soroconversão pode ser variável de indivíduo para indivíduo, podendo ser afetado pelo quadro clínico apresentado, O resultado sugere infecção com tempo de evolução inferior a 3 semanas, A capacidade protetora dos anticorpos da classe IgG não é completamente estabelecida,;',]

# Seelcionando somente os resultados que interessam
sel_exames_rapido = (data_exames_HSL['DE_EXAME'] == 'COVID-19-Teste Rápido (IgM e IgG), soro') & (data_exames_HSL[exames_covid_rapido]['DE_RESULTADO'].isin(resultados_rapido))
exames_rapido = data_exames_HSL[sel_exames_rapido]
exames_rapido.head(10) 

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
19144,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,UTI,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Anticorpos IgM, teste rápido",NÃO REAGENTE,NaN,Não reagente
19145,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,UTI,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Anticorpos IgG, teste rápido",REAGENTE,NaN,Não reagente
19146,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,UTI,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19,Antic IgM/IgG,teste rápido-Interpret",Evidência sorológica de infecção pregressa por...,NaN,NaN
19147,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,UTI,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Antic IgM/IgG, teste rápido-Observ",A dinâmica de produção de anticorpos na COVID-...,NaN,NaN
27648,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,Pronto Socorro,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Anticorpos IgM, teste rápido",NÃO REAGENTE,NaN,Não reagente
27649,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,Pronto Socorro,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Anticorpos IgG, teste rápido",NÃO REAGENTE,NaN,Não reagente
27650,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,Pronto Socorro,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19,Antic IgM/IgG,teste rápido-Interpret",Sem evidência sorológica de infecção por SARS-...,NaN,NaN
27651,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,Pronto Socorro,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Antic IgM/IgG, teste rápido-Observ",A dinâmica de produção de anticorpos na COVID-...,NaN,NaN
30956,BF74EEB9A5084C650B62388BEC2DF076,D492FEA31CBCA4F00A4C93F5C2611C18,2020-05-05,UTI,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Anticorpos IgM, teste rápido",NÃO REAGENTE,NaN,Não reagente
30957,BF74EEB9A5084C650B62388BEC2DF076,D492FEA31CBCA4F00A4C93F5C2611C18,2020-05-05,UTI,"COVID-19-Teste Rápido (IgM e IgG), soro","Covid 19, Anticorpos IgG, teste rápido",REAGENTE,NaN,Não reagente


In [143]:
# Precisamos agora mapear cada um desses resultados para detecção ou não de COVID-19:
#
# 0 => Não foram detectados anticorpos para COVID-19
# 1 => Detectados anticorpos para COVID-19
rapido_map = {
    'NÃO REAGENTE' : 0, 
    'REAGENTE' : 1,
    'Evidência sorológica de infecção pregressa por SARS-CoV-2,' : 1,
    'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, o momento da soroconversão pode ser variável de indivíduo para indivíduo, podendo ser afetado pelo quadro clínico apresentado, O resultado sugere que já tenham transcorrido mais de 3 semanas desde a infecção aguda, A capacidade protetora dos anticorpos da classe IgG não é completamente estabelecida,;' : 0,
    'Sem evidência sorológica de infecção por SARS-CoV-2,' : 0,
    'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, pode não haver anticorpos detectáveis (resultado negativo), inclusive em pessoas que tiveram infecção previamente confirmada por PCR, Se persistir a hipótese diagnóstica, sugere-se, a critério médico, repetir o teste em nova amostra colhida em 7 dias,;' : 0,
    'Evidência sorológica de infecção recente por SARS-CoV-2,' : 1,
    'A dinâmica de produção de anticorpos na COVID-19 ainda não é bem estabelecida, especialmente em pacientes que não apresentaram sintomas ou apresentaram quadro leve, Nesse sentido, o momento da soroconversão pode ser variável de indivíduo para indivíduo, podendo ser afetado pelo quadro clínico apresentado, O resultado sugere infecção com tempo de evolução inferior a 3 semanas, A capacidade protetora dos anticorpos da classe IgG não é completamente estabelecida,;' : 0
}

# Mapeando os novos valores
exames_rapido['DE_RESULTADO'] = exames_rapido['DE_RESULTADO'].map(rapido_map)
exames_rapido.drop(['DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA'], axis=1, inplace=True)
exames_rapido.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_RESULTADO
19144,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,0
19145,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,1
19146,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,1
19147,F09A86430261ABE1D630EE7090B4BC37,D1C108AC67C734A94BD2DC8A35D7E9E0,2020-06-09,0
27648,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,0
27649,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,0
27650,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,0
27651,BB8290FEA6115C206E7C89B10C5C1993,EE0BA3438A00598B875A8E4635439956,2020-05-02,0
30956,BF74EEB9A5084C650B62388BEC2DF076,D492FEA31CBCA4F00A4C93F5C2611C18,2020-05-05,0
30957,BF74EEB9A5084C650B62388BEC2DF076,D492FEA31CBCA4F00A4C93F5C2611C18,2020-05-05,1


In [144]:
# Agora analsamos quantos resultados de testes sorológicos ainda temos:
print('Após tratar os dados, temos {} testes rápidos realizados'.format(exames_rapido.shape[0]))

Após tratar os dados, temos 80 testes rápidos realizados


###**3.20) Combinando todos os testes para COVID**

In [145]:
# Podemos, finalmente, combinar todos os testes para COVID realizados em um único dataframe
exames_covid_total = pd.concat([exames_soro_total, exames_PCR_total, exames_rapido], axis=0, ignore_index=True)
exames_covid_total.drop(['ID_ATENDIMENTO'], axis=1, inplace=True)
exames_covid_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2675 entries, 0 to 2674
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID_PACIENTE   2675 non-null   object
 1   DT_COLETA     2675 non-null   object
 2   DE_RESULTADO  2675 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 62.8+ KB


In [146]:
exames_covid_total.head(20)

,ID_PACIENTE,DT_COLETA,DE_RESULTADO
0,3487791F44C34B421C932DC8616A8437,2020-05-25,1
1,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-07,1
2,C3A3D083DBC1353FAB5F87282EE86633,2020-07-23,0
3,BEBA451E2C91A4D869D14EE790FE78FD,2020-05-31,0
4,F99249272E611207DBEC7EE3CF81F0E5,2020-06-04,0
5,74D237DB6D34A5BB2A95A26CD9FD5CF9,2020-06-27,0
6,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,2020-06-26,0
7,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,2020-07-09,1
8,F09A86430261ABE1D630EE7090B4BC37,2020-06-09,0
9,C16902E84856371CFB33837851722042,2020-06-03,0


In [147]:
# Analisando quantos pacientes distintos há no conjunto de dados e quantos exames foram feitos por cada um
exames_covid_total['ID_PACIENTE'].value_counts()

C33DEE8231AD0374CAF93E52C5A473E5    17
E1DB75E3106A496D885700D3A372486D    14
3DC7E7D2D0FD41DC69FDC7DE1C0D1165    13
664A47AB7E441E661322CDBD6CC48F50    13
2C0F9E3F2F4B1705DC919E2EEA1E72E0    12
                                    ..
BCD362F20F0DDD54C5E35D82C5CBE5F7     1
BEA99816975D4154590D05CE68DA8B63     1
2FF765278F7853D96D1AD0C2DF29EF53     1
30BE2454DDCF4F58023677299EA84113     1
589083F61E15C6C8EEA0D917DF5E1A3A     1
Name: ID_PACIENTE, Length: 1662, dtype: int64

In [148]:
# Vamos exportar esse conjunto de dados temporário como um arquivo .csv
exames_covid_total.to_csv('HSL_Exames_Covid_2_EDA.csv')

##**4) Análise dos desfechos para cada paciente**

In [149]:
# Importando dados dos desfechos do arquvio 'HSL_Exames_2.csv'. O caracter de separação das colunas no arquivo .csv
# é o '|'. Exibindo os 5 primeiros dados para verificar se as informações foram importadas corretamente.
#
data_desfechos_HSL = pd.read_csv('HSL_Desfechos_2.csv', sep='|');
data_desfechos_HSL.head(5)

,id_paciente,id_atendimento,dt_atendimento,de_tipo_atendimento,id_clinica,de_clinica,dt_desfecho,de_desfecho
0,3487791F44C34B421C932DC8616A8437,33277D918111011E48FABCD6FC09012B,2020-07-08,Pronto Atendimento,6,Ortopedia,2020-07-08,Alta médica melhorado
1,3487791F44C34B421C932DC8616A8437,2DF164AD7E51B0A2F52B6DB58F904A22,2020-05-08,Externo,20,Procedimentos,2020-05-08,Alta Administrativa
2,3487791F44C34B421C932DC8616A8437,11676CFEEF6A4CDD49B0A74D3F3A85EC,2020-05-25,Externo,20,Procedimentos,2020-05-25,Alta Administrativa
3,0AD1FFA4419472256666A3445414F1F9,70A834FEFF3562FCBA3888D592DA2609,2020-03-16,Externo,20,Procedimentos,2020-03-16,Alta Administrativa
4,3487791F44C34B421C932DC8616A8437,A3F017D035922D728E06B744511A6797,2020-05-11,Pronto Atendimento,42,CL Médica Síndromes Virais,2020-05-11,Alta médica melhorado


In [150]:
# O Dataframe de desfechos não traz muitas informações relevantes sobre a detecção de COVID, mas sim sobre o 
# estado do tratamento dos pacientes. Portanto, para a aplicação do projeto 2 não serão utilizadas informações 
# deste Dataframe.

##**5) Combinando os dados em um dataframe final**

####**5.1) Combinando dataframes de exames de COVID com exames laboratoriais**

In [151]:
exames_covid_total.head(10)

,ID_PACIENTE,DT_COLETA,DE_RESULTADO
0,3487791F44C34B421C932DC8616A8437,2020-05-25,1
1,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-07,1
2,C3A3D083DBC1353FAB5F87282EE86633,2020-07-23,0
3,BEBA451E2C91A4D869D14EE790FE78FD,2020-05-31,0
4,F99249272E611207DBEC7EE3CF81F0E5,2020-06-04,0
5,74D237DB6D34A5BB2A95A26CD9FD5CF9,2020-06-27,0
6,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,2020-06-26,0
7,AA1E0A1BBB04DBFB2D4FE21BDB45FB74,2020-07-09,1
8,F09A86430261ABE1D630EE7090B4BC37,2020-06-09,0
9,C16902E84856371CFB33837851722042,2020-06-03,0


####**5.2) Adicionando os dados dos pacientes**

In [152]:
# Verificando o dataframe com os dados de sexo e idade dos pacientes
data_pacientes_HSL.head(5)

,ID_PACIENTE,IC_SEXO,IC_IDADE
0,3487791F44C34B421C932DC8616A8437,0,57.0
1,0AD1FFA4419472256666A3445414F1F9,0,51.0
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,56.0
3,962F4020D456AAB602B356E57238EF42,1,61.0
4,F983089DAC62E42124227C856AE3444C,0,73.0


In [153]:
# Verificando o dataframe com os todos os resultados dos exames de COVID-19
exames_covid_total.head(5)

,ID_PACIENTE,DT_COLETA,DE_RESULTADO
0,3487791F44C34B421C932DC8616A8437,2020-05-25,1
1,F5B5BD59D98836D8AA2D8C27B01225AB,2020-06-07,1
2,C3A3D083DBC1353FAB5F87282EE86633,2020-07-23,0
3,BEBA451E2C91A4D869D14EE790FE78FD,2020-05-31,0
4,F99249272E611207DBEC7EE3CF81F0E5,2020-06-04,0


In [154]:
# Exibindo informações sobre o tamanho de ambos os dataframes
print(data_pacientes_HSL.shape[0])
print(exames_covid_total.shape[0])

4273
2675


In [155]:
# Selecionando no dataframe de pacientes somente aqueles que realizaram testes para COVID-19
sel = data_pacientes_HSL['ID_PACIENTE'].isin(exames_covid_total['ID_PACIENTE'])
info_pacientes_teste_covid = data_pacientes_HSL[sel]
info_pacientes_teste_covid.head()

,ID_PACIENTE,IC_SEXO,IC_IDADE
0,3487791F44C34B421C932DC8616A8437,0,57.0
1,0AD1FFA4419472256666A3445414F1F9,0,51.0
2,BE35D08CF3EF6F114E9935F6D72C49FA,0,56.0
4,F983089DAC62E42124227C856AE3444C,0,73.0
8,11CE6296257A354E5D0B342F37F82111,1,66.0


In [156]:
# Unindo as informações dos pacientes que fizeram testes para COVID e o resultados dos testes em um único dataframe
pacientes_teste_covid = info_pacientes_teste_covid.merge(exames_covid_total, on=['ID_PACIENTE'], how='right')
print(pacientes_teste_covid.shape)
pacientes_teste_covid.head(20)

(2675, 5)


,ID_PACIENTE,IC_SEXO,IC_IDADE,DT_COLETA,DE_RESULTADO
0,3487791F44C34B421C932DC8616A8437,0,57.0,2020-05-25,1
1,3487791F44C34B421C932DC8616A8437,0,57.0,2020-05-25,1
2,3487791F44C34B421C932DC8616A8437,0,57.0,2020-05-25,1
3,3487791F44C34B421C932DC8616A8437,0,57.0,2020-05-25,1
4,3487791F44C34B421C932DC8616A8437,0,57.0,2020-05-04,1
5,F5B5BD59D98836D8AA2D8C27B01225AB,0,75.0,2020-06-07,1
6,F5B5BD59D98836D8AA2D8C27B01225AB,0,75.0,2020-06-07,1
7,F5B5BD59D98836D8AA2D8C27B01225AB,0,75.0,2020-06-07,1
8,F5B5BD59D98836D8AA2D8C27B01225AB,0,75.0,2020-06-07,1
9,F5B5BD59D98836D8AA2D8C27B01225AB,0,75.0,2020-05-21,1


In [157]:
# Analisando a distribuição de resultados: há uma quantidade significativa de pacientes a mais que testaram positivo 
#para COVID-19.
pacientes_teste_covid['DE_RESULTADO'].value_counts()

1    2054
0     621
Name: DE_RESULTADO, dtype: int64

####**5.3) Adicionando as features laboratoriais**

In [158]:
# Agora precisamos adicionar cada uma das features laboratoriais (que estão em dataframes individuais),
# para cada cada paciente que fez o teste de COVID-19, levando em consideração também a data de coleta,
# para termos resultados atualizados de um dos analitos que será analisado para buscar uma relação com o
# paciente ter ou não COVID-19.
#
# Unindo informações dos leucócitos dos pacientes e removendo duplicatas
merge_lab_features_1 = pacientes_teste_covid.merge(leucocitos, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_1.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_1.shape)

# Unindo informações de hemoglobina dos pacientes e removendo duplicatas
merge_lab_features_2 = merge_lab_features_1.merge(hemoglobina, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_2.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_2.shape)

# Unindo informações das plaquetas dos pacientes e removendo duplicatas
merge_lab_features_3 = merge_lab_features_2.merge(plaquetas, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_3.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_3.shape)

# Unindo informações dos neutrófilos dos pacientes e removendo duplicatas
merge_lab_features_4 = merge_lab_features_3.merge(neutrofilos_percent, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_4.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_4.shape)

# Unindo informações dos neutrófilos (%) dos pacientes e removendo duplicatas
merge_lab_features_5 = merge_lab_features_4.merge(neutrofilos, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_5.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_5.shape)

# Unindo informações dos linfócitos dos pacientes e removendo duplicatas
merge_lab_features_6 = merge_lab_features_5.merge(linfocitos_percent, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_6.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_6.shape)

# Unindo informações dos linfócitos (%) dos pacientes e removendo duplicatas
merge_lab_features_7 = merge_lab_features_6.merge(linfocitos, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_7.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_7.shape)

# Unindo informações de proteína C-reativa dos pacientes e removendo duplicatas
merge_lab_features_8 = merge_lab_features_7.merge(proteina_c, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_8.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_8.shape)

# Unindo informações de bilirrubina dos pacientes e removendo duplicatas
merge_lab_features_9 = merge_lab_features_8.merge(bilirrubina, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_9.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_9.shape)

# Unindo informações de uréia dos pacientes e removendo duplicatas
merge_lab_features_10 = merge_lab_features_9.merge(ureia, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_10.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_10.shape)

# Unindo informações de creatinina dos pacientes e removendo duplicatas
merge_lab_features_11 = merge_lab_features_10.merge(creatinina, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_11.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_11.shape)

# Unindo informações de DHL dos pacientes e removendo duplicatas
merge_lab_features_12 = merge_lab_features_11.merge(dhl, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_12.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_12.shape)

# Unindo informações dos dímeros D dos pacientes e removendo duplicatas
merge_lab_features_13 = merge_lab_features_12.merge(dimeros_d, on=['ID_PACIENTE','DT_COLETA'], how='outer')
merge_lab_features_13.drop_duplicates(subset=['ID_PACIENTE','DT_COLETA'],keep='first',inplace=True)
print(merge_lab_features_13.shape)

(6253, 6)
(6253, 7)
(6254, 8)
(6255, 9)
(6255, 10)
(6255, 11)
(6255, 12)
(6365, 13)
(6387, 14)
(6425, 15)
(6435, 16)
(6439, 17)
(6480, 18)


In [159]:
# Verificando o dataframe resultante
merge_lab_features_13.head(5)

,ID_PACIENTE,IC_SEXO,IC_IDADE,DT_COLETA,DE_RESULTADO,DE_LEUCOCITOS,DE_HEMOGLOBINA,DE_PLAQUETAS,DE_NEUTROFILOS (%),DE_NEUTROFILOS,DE_LINFOCITOS (%),DE_LINFOCITOS,DE_PROTEINA_C_REATIVA,DE_BILIRRUBINA,DE_UREIA,DE_CREATININA,DE_DHL,DE_DIMEROS_D
0,3487791F44C34B421C932DC8616A8437,0.0,57.0,2020-05-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437.374194
1,3487791F44C34B421C932DC8616A8437,0.0,57.0,2020-05-04,1.0,3390.0,14.8,86000.0,62.8,2130.0,14.5,490.0,0.19,NaN,28.0,0.91,364.0,248.000000
2,F5B5BD59D98836D8AA2D8C27B01225AB,0.0,75.0,2020-06-07,1.0,14610.0,11.8,190000.0,86.4,12620.0,7.5,1100.0,0.29,0.62,103.0,1.00,NaN,NaN
3,F5B5BD59D98836D8AA2D8C27B01225AB,0.0,75.0,2020-05-21,1.0,5000.0,12.8,202000.0,93.6,4680.0,5.6,280.0,30.67,NaN,58.0,1.29,NaN,996.000000
5,C3A3D083DBC1353FAB5F87282EE86633,1.0,32.0,2020-07-23,0.0,6690.0,13.8,203000.0,74.8,5010.0,18.1,1210.0,0.97,0.66,28.0,0.68,NaN,1437.374194


In [160]:
# Selecionamos os resultados de exame que não são nulos (resultados nulos surgem quando unimos as colunas, para os pacientes que
# não haviam realizado exame de COVID-19) e atribuimos a um novo dataframe (df_final).
sel = merge_lab_features_13['DE_RESULTADO'].notnull()
df_final = merge_lab_features_13[sel]

# Retiramos duplicatas do dataframe final
df_final.drop_duplicates(keep='first',inplace=True)

# Preenchemos valores faltantes com a média do valor da coluna, mesma estratégia adotada em (Bao, 2020)
df_final.fillna(df_final.mean(), inplace=True)

# Verificando o tamanho do dataframe final
df_final.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(2134, 18)

In [161]:
# Verificando os dados do dataframe final
df_final.head(20)

,ID_PACIENTE,IC_SEXO,IC_IDADE,DT_COLETA,DE_RESULTADO,DE_LEUCOCITOS,DE_HEMOGLOBINA,DE_PLAQUETAS,DE_NEUTROFILOS (%),DE_NEUTROFILOS,DE_LINFOCITOS (%),DE_LINFOCITOS,DE_PROTEINA_C_REATIVA,DE_BILIRRUBINA,DE_UREIA,DE_CREATININA,DE_DHL,DE_DIMEROS_D
0,3487791F44C34B421C932DC8616A8437,0.0,57.0,2020-05-25,1.0,6790.179153,13.257818,200627.035831,65.892508,4723.029316,22.709446,1364.429967,4.038129,0.437611,40.516522,1.159225,505.25731,1437.374194
1,3487791F44C34B421C932DC8616A8437,0.0,57.0,2020-05-04,1.0,3390.000000,14.800000,86000.000000,62.800000,2130.000000,14.500000,490.000000,0.190000,0.437611,28.000000,0.910000,364.00000,248.000000
2,F5B5BD59D98836D8AA2D8C27B01225AB,0.0,75.0,2020-06-07,1.0,14610.000000,11.800000,190000.000000,86.400000,12620.000000,7.500000,1100.000000,0.290000,0.620000,103.000000,1.000000,505.25731,1240.248775
3,F5B5BD59D98836D8AA2D8C27B01225AB,0.0,75.0,2020-05-21,1.0,5000.000000,12.800000,202000.000000,93.600000,4680.000000,5.600000,280.000000,30.670000,0.437611,58.000000,1.290000,505.25731,996.000000
5,C3A3D083DBC1353FAB5F87282EE86633,1.0,32.0,2020-07-23,0.0,6690.000000,13.800000,203000.000000,74.800000,5010.000000,18.100000,1210.000000,0.970000,0.660000,28.000000,0.680000,505.25731,1437.374194
6,C3A3D083DBC1353FAB5F87282EE86633,1.0,32.0,2020-03-14,0.0,6790.179153,13.257818,200627.035831,65.892508,4723.029316,22.709446,1364.429967,4.038129,0.437611,40.516522,1.159225,505.25731,1240.248775
7,BEBA451E2C91A4D869D14EE790FE78FD,0.0,48.0,2020-05-31,0.0,15060.000000,11.300000,278000.000000,92.300000,13900.000000,3.500000,530.000000,7.910000,0.437611,121.000000,2.910000,505.25731,1240.248775
8,BEBA451E2C91A4D869D14EE790FE78FD,0.0,48.0,2020-05-30,1.0,9300.000000,11.300000,267000.000000,72.300000,6720.000000,18.300000,1700.000000,2.830000,0.437611,108.000000,2.310000,505.25731,1240.248775
9,F99249272E611207DBEC7EE3CF81F0E5,1.0,78.0,2020-06-04,0.0,11620.000000,11.900000,271000.000000,74.800000,8680.000000,17.200000,2000.000000,0.530000,0.760000,84.000000,1.700000,477.00000,1334.000000
11,F99249272E611207DBEC7EE3CF81F0E5,1.0,78.0,2020-06-10,0.0,6790.179153,13.257818,200627.035831,65.892508,4723.029316,22.709446,1364.429967,4.038129,0.437611,40.516522,1.159225,505.25731,1240.248775


In [162]:
# Obtendo as informações referentes a cada coluna do dataframe final. É importante que as features que serão analisadas
# pelo algoritmo de floresta aleatória sejam numéricas. Se forem categoricas, precisam ser codificadas antes de serem
# processadas. Não é o caso no dataframe final, pois os dados foram tratados ao longo do notebook
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2134 entries, 0 to 2205
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID_PACIENTE            2134 non-null   object 
 1   IC_SEXO                2134 non-null   float64
 2   IC_IDADE               2134 non-null   float64
 3   DT_COLETA              2134 non-null   object 
 4   DE_RESULTADO           2134 non-null   float64
 5   DE_LEUCOCITOS          2134 non-null   float64
 6   DE_HEMOGLOBINA         2134 non-null   float64
 7   DE_PLAQUETAS           2134 non-null   float64
 8   DE_NEUTROFILOS (%)     2134 non-null   float64
 9   DE_NEUTROFILOS         2134 non-null   float64
 10  DE_LINFOCITOS (%)      2134 non-null   float64
 11  DE_LINFOCITOS          2134 non-null   float64
 12  DE_PROTEINA_C_REATIVA  2134 non-null   float64
 13  DE_BILIRRUBINA         2134 non-null   float64
 14  DE_UREIA               2134 non-null   float64
 15  DE_C

In [163]:
# Analisando a distribuição de resultados do dataframe final: 
# mais uma vez, há mais pacientes testando positivo do que negativo para COVID-19.
df_final['DE_RESULTADO'].value_counts()

1.0    1769
0.0     365
Name: DE_RESULTADO, dtype: int64

In [164]:
# Exportando o dataframe final para um arquivo .csv
df_final.to_csv('HSL_EDA.csv', sep='|', index=False)

In [165]:
# Verificando se é possível ler o arquivo e acessar os dados sem problemas.
df_teste = pd.read_csv('HSL_EDA.csv',sep='|')
df_teste.head(20)

,ID_PACIENTE,IC_SEXO,IC_IDADE,DT_COLETA,DE_RESULTADO,DE_LEUCOCITOS,DE_HEMOGLOBINA,DE_PLAQUETAS,DE_NEUTROFILOS (%),DE_NEUTROFILOS,DE_LINFOCITOS (%),DE_LINFOCITOS,DE_PROTEINA_C_REATIVA,DE_BILIRRUBINA,DE_UREIA,DE_CREATININA,DE_DHL,DE_DIMEROS_D
0,3487791F44C34B421C932DC8616A8437,0.0,57.0,2020-05-25,1.0,6790.179153,13.257818,200627.035831,65.892508,4723.029316,22.709446,1364.429967,4.038129,0.437611,40.516522,1.159225,505.25731,1437.374194
1,3487791F44C34B421C932DC8616A8437,0.0,57.0,2020-05-04,1.0,3390.000000,14.800000,86000.000000,62.800000,2130.000000,14.500000,490.000000,0.190000,0.437611,28.000000,0.910000,364.00000,248.000000
2,F5B5BD59D98836D8AA2D8C27B01225AB,0.0,75.0,2020-06-07,1.0,14610.000000,11.800000,190000.000000,86.400000,12620.000000,7.500000,1100.000000,0.290000,0.620000,103.000000,1.000000,505.25731,1240.248775
3,F5B5BD59D98836D8AA2D8C27B01225AB,0.0,75.0,2020-05-21,1.0,5000.000000,12.800000,202000.000000,93.600000,4680.000000,5.600000,280.000000,30.670000,0.437611,58.000000,1.290000,505.25731,996.000000
4,C3A3D083DBC1353FAB5F87282EE86633,1.0,32.0,2020-07-23,0.0,6690.000000,13.800000,203000.000000,74.800000,5010.000000,18.100000,1210.000000,0.970000,0.660000,28.000000,0.680000,505.25731,1437.374194
5,C3A3D083DBC1353FAB5F87282EE86633,1.0,32.0,2020-03-14,0.0,6790.179153,13.257818,200627.035831,65.892508,4723.029316,22.709446,1364.429967,4.038129,0.437611,40.516522,1.159225,505.25731,1240.248775
6,BEBA451E2C91A4D869D14EE790FE78FD,0.0,48.0,2020-05-31,0.0,15060.000000,11.300000,278000.000000,92.300000,13900.000000,3.500000,530.000000,7.910000,0.437611,121.000000,2.910000,505.25731,1240.248775
7,BEBA451E2C91A4D869D14EE790FE78FD,0.0,48.0,2020-05-30,1.0,9300.000000,11.300000,267000.000000,72.300000,6720.000000,18.300000,1700.000000,2.830000,0.437611,108.000000,2.310000,505.25731,1240.248775
8,F99249272E611207DBEC7EE3CF81F0E5,1.0,78.0,2020-06-04,0.0,11620.000000,11.900000,271000.000000,74.800000,8680.000000,17.200000,2000.000000,0.530000,0.760000,84.000000,1.700000,477.00000,1334.000000
9,F99249272E611207DBEC7EE3CF81F0E5,1.0,78.0,2020-06-10,0.0,6790.179153,13.257818,200627.035831,65.892508,4723.029316,22.709446,1364.429967,4.038129,0.437611,40.516522,1.159225,505.25731,1240.248775
